Webscrapping & Data Processing <br>
KARUNATHASAN Nilany <br>
SAMBATH Sïndoumady <br>
[Github Link](https://github.com/nilanyK/Webscrapping-DataProcessing)<br>
[Streamlit App Link](https://webscrapping-dataprocessing-cezgc8agfx5pjs29ct9yvg.streamlit.app/)<br>
DIA2

#### <center><font  align="center"> Project : Sustainable Nike Sneaker Marketplace 👟</font> <center>
# <center><font  align="center"> Vinted Scrapping </font> <center>
<div style="text-align:center;">
    <span style="display:inline-block; margin-right: 20px;">
        <img src="https://upload.wikimedia.org/wikipedia/commons/2/29/Vinted_logo.png" alt="Vinted Logo" width="100"/>
    </span>
</div>

___

## SUMMARY

[1. Vinted Data Scraping and Processing](#1.-Vinted-Data-Scraping-and-Processing) <br>
[2. Approximate Carbon Footprint Calculation](#2.-Approximate-Carbon-Footprint-Calculation) <br>
[3. Nike Price Scraping for Profit Calculation](#3.-Nike-Price-Scraping-for-Profit-Calculation) <br>
[4. Top 5 Most Sold Sneakers Scrapping](#4.-Top-5-Most-Sold-Sneakers-Scrapping) <br>

## 1. Vinted Data Scraping and Processing

In the following code, the method for scraping data from Vinted, with a focus on Nike brand shoes, is presented. The get_vinted_data function is structured to adapt to user preferences by incorporating parameters like gender, color, and size, allowing customization of the search based on specific criteria.

**• Mapping Dictionnaries for Filters**

Noticing changes in Vinted's filters affecting the URL structure, a flexible mapping approach is employed. This involves using dictionaries to handle these changes, ensuring accurate construction of URLs and generating personalized search queries aligned with user preferences.

User-provided gender, color, and size parameters are mapped to corresponding Vinted category IDs, ensuring precise and targeted search results. <br>
<br>
For gender-specific searches, the function utilizes mappings for men, women, girls, and boys, associating each with a unique Vinted category ID. <br>
<br>
Similarly, color and size parameters are mapped to their respective IDs using Vinted color IDs, enabling a comprehensive search. The constructed URL integrates these parameters, resulting in a refined search query that precisely aligns with the user's specified preferences. <br>



**• Navigation & Data Extraction**

The function then navigates to the constructed URL page, handling cookie acceptance to facilitate seamless interaction with the Vinted website. <br>


Iterating through paginated search results, the function retrieves essential details for each article, encompassing title, price, brand, size, link, image source, and localization. Extracting localization information involves opening a new tab for each article. To achieve this, the function navigates to the article's href link, initiating the opening of a new tab in the process. The localization information is then extracted from the details section of the article.


**• Data Processing**

Once the data is collected, the process_dataframe function is applied. This function refines and structures the raw data into a DataFrame.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd

def get_vinted_data(url, max_rows):
    driver = webdriver.Firefox()
    driver.get(url)

    # Handle cookie acceptance
    try:
        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))
        )
        cookie_button.click()
    except:
        print("Cookie button not found or timed out")

    data = {"Title": [], "Price": [], "Brand": [],  "Color" :[], "Size": [], "Link": [], "Image Source": [], "Localisation": []}

    counter = 0  # Initialize a counter to keep track of the number of rows scraped

    while counter < max_rows:
        try:
            # Wait for the articles to load on the current page
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'feed-grid'))
            )

            # Find the new articles on the current page
            article_body = driver.find_element(By.CLASS_NAME, 'feed-grid')
            articles = article_body.find_elements(By.CLASS_NAME, "feed-grid__item")

            for article in articles:
                try:
                    title_element = article.find_element(By.CLASS_NAME, 'new-item-box__overlay')
                    title = title_element.get_attribute('title')

                    # Extracting data from the title attribute
                    price_start = title.find('prix') + len('prix&nbsp;:')
                    price_end = title.find('&nbsp;€', price_start)
                    price = title[price_start:price_end].strip()

                    brand_start = title.find('marque') + len('marque&nbsp;:')
                    brand_end = title.find(',', brand_start)
                    brand = title[brand_start:brand_end].strip()

                    size_start = title.find('taille') + len('taille&nbsp;:')
                    size_end = title.find(',', size_start)
                    size = title[size_start:size_end].strip()

                    href = title_element.get_attribute('href')

                    # Extract image source from the correct elements
                    div_element = article.find_element(By.CLASS_NAME, 'new-item-box__image')
                    img_element = div_element.find_element(By.CLASS_NAME, 'web_ui__Image__content')
                    img_src = img_element.get_attribute('src')

                    # Extract localisation information from article's href link
                    driver.execute_script("window.open('', '_blank');")
                    driver.switch_to.window(driver.window_handles[1])
                    driver.get(href)

                    try:
                        localisation_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '//div[@class="details-list__item" and @data-testid="item-details-location"]/div[@class="details-list__item-value"]'))
                        )
                        
                        color_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '//div[@class="details-list__item-title" and text()="Couleur"]/following-sibling::div[@class="details-list__item-value"]'))
                    )
                        localisation = localisation_element.text
                        color = color_element.text
                    except:
                        localisation = None
                        color = None

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                    data["Title"].append(title)
                    data["Price"].append(price)
                    data["Brand"].append(brand)
                    data["Size"].append(size)
                    data["Link"].append(href)
                    data["Image Source"].append(img_src)
                    data["Localisation"].append(localisation)
                    data["Color"].append(color)
                    counter += 1 
                   

                except ValueError:
                    pass

                except StaleElementReferenceException:
                # Handle StaleElementReferenceException by reattempting to find the title element
                    continue

                if counter >= max_rows:
                    break  # Exit the loop if the counter reaches or exceeds the desired number of rows


            next_page_link = driver.find_element(By.CLASS_NAME, "web_ui__Pagination__next").get_attribute('href')

            driver.get(next_page_link)

        except NoSuchElementException:
            # If the "Next Page" is not found, break out of the loop
            break

    driver.quit()
    return pd.DataFrame(data)  # Truncate the data to the specified max_rows

def process_dataframe(df):
    data = {
        'Title': [],
        'Price': [],
        'Brand': [],
        'Color': [],
        'Size': [],
        'Link': [],
        'Image Source': [],
        'Localisation': []
    }

    for index, row in df.iterrows():
        title_components = row['Title'].split(', ')
        data['Title'].append(title_components[0])

        price_components = title_components[1].split(':')
        if len(price_components) > 1:
            price = price_components[1].strip().replace('€', '').replace('\xa0', '')
            data['Price'].append(float(price.replace(',', '.')) if price else None)
        else:
            data['Price'].append(None)

        brand_components = title_components[2].split(':')
        data['Brand'].append(brand_components[1].strip().upper() if len(brand_components) > 1 else None)
        data['Color'].append(row['Color'])
        size_components = title_components[3].split(':')
        data['Size'].append(size_components[1].strip() if len(size_components) > 1 else None)
        data['Link'].append(row['Link'])
        data['Image Source'].append(row['Image Source'])
        data['Localisation'].append(row['Localisation'])
        


    return pd.DataFrame(data)


## 2. Approximate Carbon Footprint Calculation

After extracting the data, the analysis was expanded to incorporate the environmental impact of shipping items from their respective locations to Paris. To quantify this impact, a function was developed to calculate the approximate carbon footprint associated with the localization of each article.

In the analysis, a constant emission rate per kilometer was assigned, and the carbon footprint for each item's journey to Paris was calculated. Additionally, a fixed value for Nike's carbon footprint from ordering on their website was introduced, set at 405 kg CO2 per ton (explained in our report).
<br>
The "Carbon Profit" column in the DataFrame reflects the environmental benefit attained by ordering through Vinted. It takes into account the reduction in carbon footprint compared to ordering directly from Nike's website.

In [5]:
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

# Initialize geocoder
geolocator = Nominatim(user_agent="vinted_carbon_footprint")

def calculate_approximate_carbon_footprint(location):
    try:
        # Geocode the location to get coordinates
        location_info = geolocator.geocode(location, timeout=10)

        if location_info:
            location_coordinates = (location_info.latitude, location_info.longitude)

            # Coordinates of Paris, France
            paris_coordinates = (48.8566, 2.3522)

            # Calculate the distance between the location and Paris using geopy
            distance_km = geodesic(location_coordinates, paris_coordinates).kilometers

            # Assume a constant emission rate per kilometer 
            emission_rate_per_km = 0.2  
            # Calculate the approximate carbon footprint
            carbon_footprint = distance_km * emission_rate_per_km

            return carbon_footprint

    except (ValueError, TypeError, GeocoderTimedOut):
        # Handle cases where geocoding fails or times out
        return None


## 3. Nike Price Scraping for Profit Calculation

To assess potential cost savings, a process was initiated to scrape all available prices for the specified query on the Nike website. This allows for the calculation of the average price of the desired item by aggregating the individual prices. 

In [6]:
from selenium import webdriver

from selenium.webdriver.common.by import By

def accept_cookies(driver):
    try:
        # Accept cookies
        accept_cookies_button = driver.find_element(By.CLASS_NAME, 'btn-primary-dark')
        accept_cookies_button.click()
    except:
        pass  

def get_nike_prices(url):
    driver = webdriver.Firefox()
    
    try:
        # Go to nike.fr and accept cookies
        driver.get("https://www.nike.com/fr/")
        accept_cookies(driver)

    
        # Navigate to the URL with the search query
        driver.get(url)

        # Wait for the prices to load 
        driver.implicitly_wait(10)

        # Retrieve prices
        price_elements = driver.find_elements(By.CLASS_NAME, 'product-price')
        prices = [float(price.text.replace('€', '').replace(',', '.')) for price in price_elements]

        # Calculate the average price
        average_price = sum(prices) / len(prices)

        return average_price

    except Exception as e:
        print("An error occurred:", e)

    finally:
        driver.quit()



A mechanism is implemented to calculate the potential profit made by purchasing items on Vinted compared to the average price of the same product on the Nike website. The function computes the profit made for each item. The resulting DataFrame, processed_data, is then enriched with additional columns, providing insights into the potential financial benefit of opting for Vinted over the average market price on Nike.

In [7]:
def calculate_profit_made(row, average_nike_price):
    try:
        item_price = float(row['Price'])
        profit_made = average_nike_price - item_price
        return profit_made

    except ValueError:
        return None

## 4. Top 5 Most Sold Sneakers Scrapping

### Nike Air Force 1

##### Femme

In [21]:
url = "https://www.vinted.fr/catalog?search_text=nike%20air%20force%201&catalog[]=16&brand_ids[]=53&status_ids[]=6&status_ids[]=1&status_ids[]=2"
vinted_data_airforce1_femme = get_vinted_data(url,1017)
df_nike_airforce1_Femme = process_dataframe(vinted_data_airforce1_femme)
df_nike_airforce1_Femme


,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Nike air force 1 jester xx light cream ghost a...,45.0,NIKE,"CRÈME, TURQUOISE",36.5,https://www.vinted.fr/items/3884190992-nike-ai...,https://images1.vinted.net/t/02_02216_nyMm5THq...,"ARNHEM, PAYS-BAS"
1,Air force 1,100.0,NIKE,MULTICOLORE,40,https://www.vinted.fr/items/3884192255-air-for...,https://images1.vinted.net/t/02_01474_n1ZAFHdk...,"SAINT-ÉTIENNE, FRANCE"
2,Air Force 1,60.0,NIKE,BLANC,40.5,https://www.vinted.fr/items/3884110320-air-for...,https://images1.vinted.net/t/01_00c00_NQGqWa1v...,FRANCE
3,Nike air force 1 fontanka,84.0,NIKE,BLANC,39,https://www.vinted.fr/items/3884099583-nike-ai...,https://images1.vinted.net/t/02_01dfd_v7uyx1SD...,ITALIE
4,Air force 1 customisé,45.0,NIKE,"NOIR, BLANC",36,https://www.vinted.fr/items/3884036184-air-for...,https://images1.vinted.net/t/03_01b8d_nFsrDmMp...,"CORBAS, FRANCE"
...,...,...,...,...,...,...,...,...
1012,Basket Nike Air Force 1,10.0,NIKE,BORDEAUX,37.5,https://www.vinted.fr/items/3855241405-basket-...,https://images1.vinted.net/t/02_00396_KMZuoKYp...,"SIMIANE-COLLONGUE, FRANCE"
1013,"Nike Air Force 1 T36,5",35.0,NIKE,NOIR,36.5,https://www.vinted.fr/items/3855185427-nike-ai...,https://images1.vinted.net/t/01_010a1_QoSPToec...,"SAINT-DENIS, FRANCE"
1014,Basket Nike air force 1 39,45.0,NIKE,"NOIR, DORÉ",39,https://www.vinted.fr/items/3855163605-basket-...,https://images1.vinted.net/t/03_01fa3_JyfyPd1j...,"VAUDÉVILLE, FRANCE"
1015,air force 1 MID,10.0,NIKE,BLANC,37.5,https://www.vinted.fr/items/3855083727-air-for...,https://images1.vinted.net/t/03_0168d_cYv84mGg...,"NAPOLI, ITALIE"


In [52]:
df_nike_airforce1_Femme["Gender"]="Women"
df_nike_airforce1_Femme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,Air force 1,25.0,NIKE,NOIR,44,https://www.vinted.fr/items/3832363233-air-for...,https://images1.vinted.net/t/01_01c6e_SYGW7Lxz...,FRANCE,Women
1,Nike Air Force 1,9.0,NIKE,BLANC,"38,5",https://www.vinted.fr/items/3850051271-nike-ai...,https://images1.vinted.net/t/03_02311_bcKpTshh...,"MADRID, ESPAGNE",Women
2,Nike AIR Force 1,46.0,NIKE,NOIR,43,https://www.vinted.fr/items/3882316593-nike-ai...,https://images1.vinted.net/t/03_02701_SXDAUzVh...,"ROUEN, FRANCE",Women
3,Air force 1 white,45.0,NIKE,BLANC,42,https://www.vinted.fr/items/3882379404-air-for...,https://images1.vinted.net/t/01_00e41_icqCaUTS...,"PISTOIA, ITALIE",Women
4,Nike Air Force 1,25.0,NIKE,"GRIS, BLANC",43,https://www.vinted.fr/items/3745924511-nike-ai...,https://images1.vinted.net/t/02_00fd2_8BxiRrhS...,"GALLIERA VENETA, ITALIE",Women
...,...,...,...,...,...,...,...,...,...
968,Nike air force 1,70.0,NIKE,BLANC,42,https://www.vinted.fr/items/222500700-nike-air...,https://images1.vinted.net/t/01_0263f_R8GM8kFc...,"MOULINS, FRANCE",Women
969,NIKE Air Force 1,240.0,NIKE,"NOIR, JAUNE",43,https://www.vinted.fr/items/1341324493-nike-ai...,https://images1.vinted.net/t/01_0230f_D1KtpVjE...,FRANCE,Women
970,Nike Air force 1,70.0,NIKE,"BLEU, BLANC","44,5",https://www.vinted.fr/items/284967393-nike-air...,https://images1.vinted.net/t/01_014d9_fJmh747T...,"ROTTERDAM, PAYS-BAS",Women
971,Air force 1,120.0,NIKE,BLANC,43,https://www.vinted.fr/items/2400514169-air-for...,https://images1.vinted.net/t/03_00275_r6zStbHK...,"ROME, ITALIE",Women


In [53]:
df_nike_airforce1_Femme.to_csv("data_NikeAirForce1_FEMME.csv", index=False)

##### Homme

In [27]:
url = "https://www.vinted.fr/catalog?search_text=nike%20air%20force%201&catalog[]=1231&brand_ids[]=53&status_ids[]=6&status_ids[]=1&status_ids[]=2"
vinted_data_airforce1_homme = get_vinted_data(url,973)
df_nike_airforce1_Homme = process_dataframe(vinted_data_airforce1_homme)
df_nike_airforce1_Homme


,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Air force 1,25.0,NIKE,NOIR,44,https://www.vinted.fr/items/3832363233-air-for...,https://images1.vinted.net/t/01_01c6e_SYGW7Lxz...,FRANCE
1,Nike Air Force 1,9.0,NIKE,BLANC,"38,5",https://www.vinted.fr/items/3850051271-nike-ai...,https://images1.vinted.net/t/03_02311_bcKpTshh...,"MADRID, ESPAGNE"
2,Nike AIR Force 1,46.0,NIKE,NOIR,43,https://www.vinted.fr/items/3882316593-nike-ai...,https://images1.vinted.net/t/03_02701_SXDAUzVh...,"ROUEN, FRANCE"
3,Air force 1 white,45.0,NIKE,BLANC,42,https://www.vinted.fr/items/3882379404-air-for...,https://images1.vinted.net/t/01_00e41_icqCaUTS...,"PISTOIA, ITALIE"
4,Nike Air Force 1,25.0,NIKE,"GRIS, BLANC",43,https://www.vinted.fr/items/3745924511-nike-ai...,https://images1.vinted.net/t/02_00fd2_8BxiRrhS...,"GALLIERA VENETA, ITALIE"
...,...,...,...,...,...,...,...,...
968,Nike air force 1,70.0,NIKE,BLANC,42,https://www.vinted.fr/items/222500700-nike-air...,https://images1.vinted.net/t/01_0263f_R8GM8kFc...,"MOULINS, FRANCE"
969,NIKE Air Force 1,240.0,NIKE,"NOIR, JAUNE",43,https://www.vinted.fr/items/1341324493-nike-ai...,https://images1.vinted.net/t/01_0230f_D1KtpVjE...,FRANCE
970,Nike Air force 1,70.0,NIKE,"BLEU, BLANC","44,5",https://www.vinted.fr/items/284967393-nike-air...,https://images1.vinted.net/t/01_014d9_fJmh747T...,"ROTTERDAM, PAYS-BAS"
971,Air force 1,120.0,NIKE,BLANC,43,https://www.vinted.fr/items/2400514169-air-for...,https://images1.vinted.net/t/03_00275_r6zStbHK...,"ROME, ITALIE"


In [29]:
df_nike_airforce1_Homme['Gender']="Men"
df_nike_airforce1_Homme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,Air force 1,25.0,NIKE,NOIR,44,https://www.vinted.fr/items/3832363233-air-for...,https://images1.vinted.net/t/01_01c6e_SYGW7Lxz...,FRANCE,Men
1,Nike Air Force 1,9.0,NIKE,BLANC,"38,5",https://www.vinted.fr/items/3850051271-nike-ai...,https://images1.vinted.net/t/03_02311_bcKpTshh...,"MADRID, ESPAGNE",Men
2,Nike AIR Force 1,46.0,NIKE,NOIR,43,https://www.vinted.fr/items/3882316593-nike-ai...,https://images1.vinted.net/t/03_02701_SXDAUzVh...,"ROUEN, FRANCE",Men
3,Air force 1 white,45.0,NIKE,BLANC,42,https://www.vinted.fr/items/3882379404-air-for...,https://images1.vinted.net/t/01_00e41_icqCaUTS...,"PISTOIA, ITALIE",Men
4,Nike Air Force 1,25.0,NIKE,"GRIS, BLANC",43,https://www.vinted.fr/items/3745924511-nike-ai...,https://images1.vinted.net/t/02_00fd2_8BxiRrhS...,"GALLIERA VENETA, ITALIE",Men
...,...,...,...,...,...,...,...,...,...
968,Nike air force 1,70.0,NIKE,BLANC,42,https://www.vinted.fr/items/222500700-nike-air...,https://images1.vinted.net/t/01_0263f_R8GM8kFc...,"MOULINS, FRANCE",Men
969,NIKE Air Force 1,240.0,NIKE,"NOIR, JAUNE",43,https://www.vinted.fr/items/1341324493-nike-ai...,https://images1.vinted.net/t/01_0230f_D1KtpVjE...,FRANCE,Men
970,Nike Air force 1,70.0,NIKE,"BLEU, BLANC","44,5",https://www.vinted.fr/items/284967393-nike-air...,https://images1.vinted.net/t/01_014d9_fJmh747T...,"ROTTERDAM, PAYS-BAS",Men
971,Air force 1,120.0,NIKE,BLANC,43,https://www.vinted.fr/items/2400514169-air-for...,https://images1.vinted.net/t/03_00275_r6zStbHK...,"ROME, ITALIE",Men


In [34]:
df_nike_airforce1_Homme.to_csv("data_NikeAirForce1_HOMME.csv",index=False)

##### Enfant

In [35]:
url = "https://www.vinted.fr/catalog?search_text=nike%20air%20force%201&catalog[]=1193&brand_ids[]=53&status_ids[]=6&status_ids[]=1&status_ids[]=2"
vinted_data_airforce1_enfant = get_vinted_data(url,121)
df_nike_airforce1_Enfant = process_dataframe(vinted_data_airforce1_enfant)
df_nike_airforce1_Enfant


,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Nike air force 1,30.8,NIKE,"BLANC, VERT",39,https://www.vinted.fr/items/3567224229-nike-ai...,https://images1.vinted.net/t/02_00853_4G8xWkub...,"LYON, FRANCE"
1,Air force 1,10.0,NIKE,ROUGE,23,https://www.vinted.fr/items/3863106013-air-for...,https://images1.vinted.net/t/03_0204e_vU2TWFHH...,FRANCE
2,Baskets bébé Nike Air Force 1 taille 18,23.0,NIKE,BLANC,18,https://www.vinted.fr/items/3689480737-baskets...,https://images1.vinted.net/t/01_0191f_LAXJvQvj...,FRANCE
3,Air force,15.0,NIKE,BLANC,Jusqu'à 1 mois / 50 cm,https://www.vinted.fr/items/3860844023-air-for...,https://images1.vinted.net/t/03_00df7_xiWG3gxM...,"LE MANS, FRANCE"
4,Air force 1,6.0,NIKE,"NOIR, BLANC",22,https://www.vinted.fr/items/3573625925-air-for...,https://images1.vinted.net/t/03_02348_iYtALgyu...,"MONTPELLIER, FRANCE"
...,...,...,...,...,...,...,...,...
117,Nike air force 1,30.0,NIKE,"NOIR, BLANC",22,https://www.vinted.fr/items/3767520836-nike-ai...,https://images1.vinted.net/t/01_01dec_iQrvpg4D...,"CHÂTEAUROUX, FRANCE"
118,Air force 1,25.0,NIKE,BLANC,16,https://www.vinted.fr/items/3486491198-air-for...,https://images1.vinted.net/t/03_009a5_ZBsnvYCP...,"CRESPIN, FRANCE"
119,Nike air force 1,15.0,NIKE,BLANC,27,https://www.vinted.fr/items/3825764645-nike-ai...,https://images1.vinted.net/t/01_005ea_b26Bec5S...,"VACOGNES-NEUILLY, FRANCE"
120,Nike air force 1,17.0,NIKE,"BLANC, ORANGE",29,https://www.vinted.fr/items/3416257001-nike-ai...,https://images1.vinted.net/t/01_02066_dA53nsng...,"ROYE, FRANCE"


In [36]:
df_nike_airforce1_Enfant['Gender']="Kid"
df_nike_airforce1_Enfant

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,Nike air force 1,30.8,NIKE,"BLANC, VERT",39,https://www.vinted.fr/items/3567224229-nike-ai...,https://images1.vinted.net/t/02_00853_4G8xWkub...,"LYON, FRANCE",Kid
1,Air force 1,10.0,NIKE,ROUGE,23,https://www.vinted.fr/items/3863106013-air-for...,https://images1.vinted.net/t/03_0204e_vU2TWFHH...,FRANCE,Kid
2,Baskets bébé Nike Air Force 1 taille 18,23.0,NIKE,BLANC,18,https://www.vinted.fr/items/3689480737-baskets...,https://images1.vinted.net/t/01_0191f_LAXJvQvj...,FRANCE,Kid
3,Air force,15.0,NIKE,BLANC,Jusqu'à 1 mois / 50 cm,https://www.vinted.fr/items/3860844023-air-for...,https://images1.vinted.net/t/03_00df7_xiWG3gxM...,"LE MANS, FRANCE",Kid
4,Air force 1,6.0,NIKE,"NOIR, BLANC",22,https://www.vinted.fr/items/3573625925-air-for...,https://images1.vinted.net/t/03_02348_iYtALgyu...,"MONTPELLIER, FRANCE",Kid
...,...,...,...,...,...,...,...,...,...
117,Nike air force 1,30.0,NIKE,"NOIR, BLANC",22,https://www.vinted.fr/items/3767520836-nike-ai...,https://images1.vinted.net/t/01_01dec_iQrvpg4D...,"CHÂTEAUROUX, FRANCE",Kid
118,Air force 1,25.0,NIKE,BLANC,16,https://www.vinted.fr/items/3486491198-air-for...,https://images1.vinted.net/t/03_009a5_ZBsnvYCP...,"CRESPIN, FRANCE",Kid
119,Nike air force 1,15.0,NIKE,BLANC,27,https://www.vinted.fr/items/3825764645-nike-ai...,https://images1.vinted.net/t/01_005ea_b26Bec5S...,"VACOGNES-NEUILLY, FRANCE",Kid
120,Nike air force 1,17.0,NIKE,"BLANC, ORANGE",29,https://www.vinted.fr/items/3416257001-nike-ai...,https://images1.vinted.net/t/01_02066_dA53nsng...,"ROYE, FRANCE",Kid


In [38]:
df_nike_airforce1_Enfant.to_csv("data_NikeAirForce1_ENFANT.csv",index=False)

##### Concataned Dataframe of Nike Air Force 1

In [54]:
df_Femme = pd.read_csv("data_NikeAirForce1_FEMME.csv",sep=",")
df_Homme = pd.read_csv("data_NikeAirForce1_HOMME.csv",sep=",")
df_Enfant = pd.read_csv("data_NikeAirForce1_ENFANT.csv",sep=",")

In [55]:
df_nike_airforce1 = pd.concat([df_Femme, df_Homme, df_Enfant], axis=0, ignore_index=True)
df_nike_airforce1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,Air force 1,25.0,NIKE,NOIR,44,https://www.vinted.fr/items/3832363233-air-for...,https://images1.vinted.net/t/01_01c6e_SYGW7Lxz...,FRANCE,Women
1,Nike Air Force 1,9.0,NIKE,BLANC,"38,5",https://www.vinted.fr/items/3850051271-nike-ai...,https://images1.vinted.net/t/03_02311_bcKpTshh...,"MADRID, ESPAGNE",Women
2,Nike AIR Force 1,46.0,NIKE,NOIR,43,https://www.vinted.fr/items/3882316593-nike-ai...,https://images1.vinted.net/t/03_02701_SXDAUzVh...,"ROUEN, FRANCE",Women
3,Air force 1 white,45.0,NIKE,BLANC,42,https://www.vinted.fr/items/3882379404-air-for...,https://images1.vinted.net/t/01_00e41_icqCaUTS...,"PISTOIA, ITALIE",Women
4,Nike Air Force 1,25.0,NIKE,"GRIS, BLANC",43,https://www.vinted.fr/items/3745924511-nike-ai...,https://images1.vinted.net/t/02_00fd2_8BxiRrhS...,"GALLIERA VENETA, ITALIE",Women
...,...,...,...,...,...,...,...,...,...
2063,Nike air force 1,30.0,NIKE,"NOIR, BLANC",22,https://www.vinted.fr/items/3767520836-nike-ai...,https://images1.vinted.net/t/01_01dec_iQrvpg4D...,"CHÂTEAUROUX, FRANCE",Kid
2064,Air force 1,25.0,NIKE,BLANC,16,https://www.vinted.fr/items/3486491198-air-for...,https://images1.vinted.net/t/03_009a5_ZBsnvYCP...,"CRESPIN, FRANCE",Kid
2065,Nike air force 1,15.0,NIKE,BLANC,27,https://www.vinted.fr/items/3825764645-nike-ai...,https://images1.vinted.net/t/01_005ea_b26Bec5S...,"VACOGNES-NEUILLY, FRANCE",Kid
2066,Nike air force 1,17.0,NIKE,"BLANC, ORANGE",29,https://www.vinted.fr/items/3416257001-nike-ai...,https://images1.vinted.net/t/01_02066_dA53nsng...,"ROYE, FRANCE",Kid


In [57]:
# Adding columns to the DataFrame for the approximate carbon footprint and Nike's fixed value
df_nike_airforce1["Approximate Carbon Footprint (kg CO2)"] = df_nike_airforce1["Localisation"].apply(calculate_approximate_carbon_footprint)
df_nike_airforce1["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] = 405  

# Adding a column for Carbon Profit
df_nike_airforce1["Carbon Profit (kg CO2)"] = df_nike_airforce1["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] - df_nike_airforce1["Approximate Carbon Footprint (kg CO2)"]

df_nike_airforce1


,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2)
0,Air force 1,25.0,NIKE,NOIR,44,https://www.vinted.fr/items/3832363233-air-for...,https://images1.vinted.net/t/01_01c6e_SYGW7Lxz...,FRANCE,Women,50.586158,405,354.413842
1,Nike Air Force 1,9.0,NIKE,BLANC,"38,5",https://www.vinted.fr/items/3850051271-nike-ai...,https://images1.vinted.net/t/03_02311_bcKpTshh...,"MADRID, ESPAGNE",Women,210.593392,405,194.406608
2,Nike AIR Force 1,46.0,NIKE,NOIR,43,https://www.vinted.fr/items/3882316593-nike-ai...,https://images1.vinted.net/t/03_02701_SXDAUzVh...,"ROUEN, FRANCE",Women,22.487025,405,382.512975
3,Air force 1 white,45.0,NIKE,BLANC,42,https://www.vinted.fr/items/3882379404-air-for...,https://images1.vinted.net/t/01_00e41_icqCaUTS...,"PISTOIA, ITALIE",Women,169.910823,405,235.089177
4,Nike Air Force 1,25.0,NIKE,"GRIS, BLANC",43,https://www.vinted.fr/items/3745924511-nike-ai...,https://images1.vinted.net/t/02_00fd2_8BxiRrhS...,"GALLIERA VENETA, ITALIE",Women,159.906946,405,245.093054
...,...,...,...,...,...,...,...,...,...,...,...,...
2063,Nike air force 1,30.0,NIKE,"NOIR, BLANC",22,https://www.vinted.fr/items/3767520836-nike-ai...,https://images1.vinted.net/t/01_01dec_iQrvpg4D...,"CHÂTEAUROUX, FRANCE",Kid,46.394157,405,358.605843
2064,Air force 1,25.0,NIKE,BLANC,16,https://www.vinted.fr/items/3486491198-air-for...,https://images1.vinted.net/t/03_009a5_ZBsnvYCP...,"CRESPIN, FRANCE",Kid,39.542811,405,365.457189
2065,Nike air force 1,15.0,NIKE,BLANC,27,https://www.vinted.fr/items/3825764645-nike-ai...,https://images1.vinted.net/t/01_005ea_b26Bec5S...,"VACOGNES-NEUILLY, FRANCE",Kid,42.422550,405,362.577450
2066,Nike air force 1,17.0,NIKE,"BLANC, ORANGE",29,https://www.vinted.fr/items/3416257001-nike-ai...,https://images1.vinted.net/t/01_02066_dA53nsng...,"ROYE, FRANCE",Kid,19.797425,405,385.202575


In [63]:
url1 = "https://www.nike.com/fr/w/air-force-1-chaussures-5sj3yzy7ok"
average_nike_price1=get_nike_prices(url1)
average_nike_price1

117.33137931034476

In [65]:
df_nike_airforce1.sort_values(by='Price', inplace=True)

# Calculate profit made for Vinted data
df_nike_airforce1['Average price of the product on Nike'] = average_nike_price1
df_nike_airforce1['Profit Made'] = df_nike_airforce1.apply(lambda row: calculate_profit_made(row, average_nike_price1), axis=1)

df_nike_airforce1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
2057,Air force 1,3.5,NIKE,"NOIR, BLANC",23,https://www.vinted.fr/items/3803206036-air-for...,https://images1.vinted.net/t/03_00e3f_soGZYV6Z...,"PORTO, PORTUGAL",Kid,242.903503,405,162.096497,117.331379,113.831379
1959,Chaussures Nike Air Orange,3.5,NIKE,"ORANGE, BLANC",37,https://www.vinted.fr/items/3124028095-chaussu...,https://images1.vinted.net/t/01_02588_5vc8Nbbr...,"SAINTE-MAURE-DE-TOURAINE, FRANCE",Kid,46.653032,405,358.346968,117.331379,113.831379
1969,Air force bébé,5.0,NIKE,BLANC,1-3 mois / 56 cm,https://www.vinted.fr/items/3649411494-air-for...,https://images1.vinted.net/t/03_026e1_1gfWBBDD...,"NORMANVILLE, FRANCE",Kid,31.657668,405,373.342332,117.331379,112.331379
992,Nike Air Force 1,5.0,NIKE,"BLEU, JAUNE","40,5",https://www.vinted.fr/items/3639577938-nike-ai...,https://images1.vinted.net/t/02_023d5_t8rfdspW...,"BILBAO, ESPAGNE",Men,148.770908,405,256.229092,117.331379,112.331379
1983,Air Force 1,5.0,NIKE,BLANC,18,https://www.vinted.fr/items/3843477416-air-for...,https://images1.vinted.net/t/02_023e4_eML69SVV...,"FIGARI, FRANCE",Kid,195.212950,405,209.787050,117.331379,112.331379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,Air Force 1,NaN,"50,00 €","ROUGE, BLANC",Nike,https://www.vinted.fr/items/979490332-air-forc...,https://images1.vinted.net/t/01_0214b_PqZ2pE75...,ESPAGNE,Women,240.819687,405,164.180313,117.331379,NaN
1095,Zapatillas Nike Air Force 1,NaN,"100,00 €","BLANC, ROUGE",Nike,https://www.vinted.fr/items/3880451674-zapatil...,https://images1.vinted.net/t/01_00201_RWnrBy4x...,"CORDOBA, ESPAGNE",Men,269.499901,405,135.500099,117.331379,NaN
1120,Zapatillas Nike Air Force 1 x Supreme,NaN,"110,00 €",BLANC,Nike,https://www.vinted.fr/items/3880460837-zapatil...,https://images1.vinted.net/t/01_02708_LbjX5Yfs...,"CORDOBA, ESPAGNE",Men,269.499901,405,135.500099,117.331379,NaN
1896,Air Force 1,NaN,"50,00 €","ROUGE, BLANC",Nike,https://www.vinted.fr/items/979490332-air-forc...,https://images1.vinted.net/t/01_0214b_PqZ2pE75...,ESPAGNE,Men,240.819687,405,164.180313,117.331379,NaN


In [67]:
df_nike_airforce1[df_nike_airforce1['Profit Made'].isna()]

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
122,Zapatillas Nike Air Force 1,NaN,"100,00 €","BLANC, ROUGE",Nike,https://www.vinted.fr/items/3880451674-zapatil...,https://images1.vinted.net/t/01_00201_RWnrBy4x...,"CORDOBA, ESPAGNE",Women,269.499901,405,135.500099,117.331379,NaN
147,Zapatillas Nike Air Force 1 x Supreme,NaN,"110,00 €",BLANC,Nike,https://www.vinted.fr/items/3880460837-zapatil...,https://images1.vinted.net/t/01_02708_LbjX5Yfs...,"CORDOBA, ESPAGNE",Women,269.499901,405,135.500099,117.331379,NaN
923,Air Force 1,NaN,"50,00 €","ROUGE, BLANC",Nike,https://www.vinted.fr/items/979490332-air-forc...,https://images1.vinted.net/t/01_0214b_PqZ2pE75...,ESPAGNE,Women,240.819687,405,164.180313,117.331379,NaN
1095,Zapatillas Nike Air Force 1,NaN,"100,00 €","BLANC, ROUGE",Nike,https://www.vinted.fr/items/3880451674-zapatil...,https://images1.vinted.net/t/01_00201_RWnrBy4x...,"CORDOBA, ESPAGNE",Men,269.499901,405,135.500099,117.331379,NaN
1120,Zapatillas Nike Air Force 1 x Supreme,NaN,"110,00 €",BLANC,Nike,https://www.vinted.fr/items/3880460837-zapatil...,https://images1.vinted.net/t/01_02708_LbjX5Yfs...,"CORDOBA, ESPAGNE",Men,269.499901,405,135.500099,117.331379,NaN
1896,Air Force 1,NaN,"50,00 €","ROUGE, BLANC",Nike,https://www.vinted.fr/items/979490332-air-forc...,https://images1.vinted.net/t/01_0214b_PqZ2pE75...,ESPAGNE,Men,240.819687,405,164.180313,117.331379,NaN
1964,👟 Basket Nike Air Force 1,NaN,"15,00 €","BEIGE, ROSE",Nike,https://www.vinted.fr/items/3862369041-basket-...,https://images1.vinted.net/t/03_01cbb_sHsiCHCn...,FRANCE,Kid,50.586158,405,354.413842,117.331379,NaN


In [68]:
df_nike_airforce1_cleaned= df_nike_airforce1.dropna(subset=['Profit Made'])

In [69]:
df_nike_airforce1_cleaned

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
2057,Air force 1,3.5,NIKE,"NOIR, BLANC",23,https://www.vinted.fr/items/3803206036-air-for...,https://images1.vinted.net/t/03_00e3f_soGZYV6Z...,"PORTO, PORTUGAL",Kid,242.903503,405,162.096497,117.331379,113.831379
1959,Chaussures Nike Air Orange,3.5,NIKE,"ORANGE, BLANC",37,https://www.vinted.fr/items/3124028095-chaussu...,https://images1.vinted.net/t/01_02588_5vc8Nbbr...,"SAINTE-MAURE-DE-TOURAINE, FRANCE",Kid,46.653032,405,358.346968,117.331379,113.831379
1969,Air force bébé,5.0,NIKE,BLANC,1-3 mois / 56 cm,https://www.vinted.fr/items/3649411494-air-for...,https://images1.vinted.net/t/03_026e1_1gfWBBDD...,"NORMANVILLE, FRANCE",Kid,31.657668,405,373.342332,117.331379,112.331379
992,Nike Air Force 1,5.0,NIKE,"BLEU, JAUNE","40,5",https://www.vinted.fr/items/3639577938-nike-ai...,https://images1.vinted.net/t/02_023d5_t8rfdspW...,"BILBAO, ESPAGNE",Men,148.770908,405,256.229092,117.331379,112.331379
1983,Air Force 1,5.0,NIKE,BLANC,18,https://www.vinted.fr/items/3843477416-air-for...,https://images1.vinted.net/t/02_023e4_eML69SVV...,"FIGARI, FRANCE",Kid,195.212950,405,209.787050,117.331379,112.331379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1372,Air force 1 éditions limité.,220.0,NIKE,ROUGE,40,https://www.vinted.fr/items/3880468290-air-for...,https://images1.vinted.net/t/01_00a5f_DexER3Wd...,FRANCE,Men,50.586158,405,354.413842,117.331379,-102.668621
969,NIKE Air Force 1,240.0,NIKE,"NOIR, JAUNE",43,https://www.vinted.fr/items/1341324493-nike-ai...,https://images1.vinted.net/t/01_0230f_D1KtpVjE...,FRANCE,Women,50.586158,405,354.413842,117.331379,-122.668621
1942,NIKE Air Force 1,240.0,NIKE,"NOIR, JAUNE",43,https://www.vinted.fr/items/1341324493-nike-ai...,https://images1.vinted.net/t/01_0230f_D1KtpVjE...,FRANCE,Men,50.586158,405,354.413842,117.331379,-122.668621
1209,Nike Air Force 1 Snakeskin,280.0,NIKE,BEIGE,43,https://www.vinted.fr/items/3877283597-nike-ai...,https://images1.vinted.net/t/01_003b9_kwUzPjNY...,"PARIS, FRANCE",Men,0.088844,405,404.911156,117.331379,-162.668621


In [70]:
df_nike_airforce1_cleaned.to_csv("AllData_Nike_AirForce1.csv",index=False)

### Nike Blazer

##### Femme

In [74]:
url = "https://www.vinted.fr/catalog?search_text=Nike%20Blazer&brand_ids[]=53&status_ids[]=6&status_ids[]=1&status_ids[]=2&catalog[]=16"
vinted_data_blazer_femme = get_vinted_data(url,1017)
df_nike_blazer_Femme = process_dataframe(vinted_data_blazer_femme)
df_nike_blazer_Femme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Nike blazer,18.0,NIKE,"NOIR, BLANC",38,https://www.vinted.fr/items/3876910512-nike-bl...,https://images1.vinted.net/t/03_01ead_Qq9rWDqE...,FRANCE
1,Nike Blazer,20.0,NIKE,"NOIR, BLANC",35,https://www.vinted.fr/items/3848826530-nike-bl...,https://images1.vinted.net/t/01_008bd_rpQ6BAxe...,"BARCELONA, ESPAGNE"
2,Nike blazer,8.0,NIKE,"NOIR, BLANC",38.5,https://www.vinted.fr/items/3221377177-nike-bl...,https://images1.vinted.net/t/01_007ea_Vye88dkb...,"MESSINA, ITALIE"
3,Nike blazer,10.0,NIKE,"BLANC, JAUNE",37.5,https://www.vinted.fr/items/3148333128-nike-bl...,https://images1.vinted.net/t/02_01b0c_E4Rk5HB1...,ITALIE
4,Nike Blazer,14.9,NIKE,"BLANC, MARINE",40,https://www.vinted.fr/items/3884765098-nike-bl...,https://images1.vinted.net/t/01_002c1_cctsdgjk...,PORTUGAL
...,...,...,...,...,...,...,...,...
1012,Blazer Nike,10.0,NIKE,MOUTARDE,38.5,https://www.vinted.fr/items/3835803251-blazer-...,https://images1.vinted.net/t/01_00841_mWXkYUbC...,"ISTRANA, ITALIE"
1013,Nike blazer,30.0,NIKE,"NOIR, BLANC",35,https://www.vinted.fr/items/3503583077-nike-bl...,https://images1.vinted.net/t/03_007f5_JfJXc28J...,"AIX-EN-PROVENCE, FRANCE"
1014,nike blazer,32.0,NIKE,"NOIR, BLANC",41,https://www.vinted.fr/items/3352969201-nike-bl...,https://images1.vinted.net/t/01_01663_CNdPd6Bz...,"SAINT-JEAN-D'ANGÉLY, FRANCE"
1015,Nike blazer,12.0,NIKE,VIOLET,40,https://www.vinted.fr/items/998626394-nike-bla...,https://images1.vinted.net/t/02_004cd_kgYCBDmU...,"ALESSANDRIA, ITALIE"


In [79]:
df_nike_blazer_Femme['Gender']="Women"

In [80]:
df_nike_blazer_Femme.to_csv("data_NikeBlazer_FEMME.csv",index=False)

##### Homme

In [81]:
url = "https://www.vinted.fr/catalog?search_text=Nike%20Blazer&brand_ids[]=53&status_ids[]=6&status_ids[]=1&status_ids[]=2&catalog[]=1231"
vinted_data_blazer_homme = get_vinted_data(url,1017)
df_nike_blazer_Homme= process_dataframe(vinted_data_blazer_homme)
df_nike_blazer_Homme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Nike blazer,20.0,NIKE,"BLANC, BLEU",38,https://www.vinted.fr/items/3867848764-nike-bl...,https://images1.vinted.net/t/03_00dde_kY54EGQi...,"ZEELAND, PAYS-BAS"
1,Nike blazer mid 77 pro club,40.0,NIKE,BLANC,42,https://www.vinted.fr/items/3879733803-nike-bl...,https://images1.vinted.net/t/03_01ee7_w1L1jBhc...,"BARCELONA, ESPAGNE"
2,Nike blazer,9.0,NIKE,"GRIS, BEIGE",39,https://www.vinted.fr/items/2828630169-nike-bl...,https://images1.vinted.net/t/02_0085b_uBkPc6wt...,"DIJON, FRANCE"
3,Blazer Nike,25.0,NIKE,"NOIR, BLANC","38,5",https://www.vinted.fr/items/3863283938-blazer-...,https://images1.vinted.net/t/03_01e07_R8oTfmLZ...,FRANCE
4,Blazer Nike,24.0,NIKE,BLANC,38,https://www.vinted.fr/items/3837057054-blazer-...,https://images1.vinted.net/t/01_02440_NUQMbm83...,"ROQUEMAURE, FRANCE"
...,...,...,...,...,...,...,...,...
1012,Nike blazer,15.0,NIKE,BLANC,42,https://www.vinted.fr/items/2862292959-nike-bl...,https://images1.vinted.net/t/03_02031_42dPu6Tc...,ITALIE
1013,Nike blazer,35.0,NIKE,"GRIS, BLANC",42,https://www.vinted.fr/items/2405452965-nike-bl...,https://images1.vinted.net/t/02_01d29_uLmJgm2W...,"LA CANONJA, ESPAGNE"
1014,blazer nike,40.0,NIKE,"NOIR, BLANC",43,https://www.vinted.fr/items/1455889141-blazer-...,https://images1.vinted.net/t/03_016e3_o9Q1VR9v...,"PAGANI, ITALIE"
1015,Nike blazer,40.0,NIKE,"NOIR, BLANC",41,https://www.vinted.fr/items/2192244789-nike-bl...,https://images1.vinted.net/t/02_00c3c_JPuGs1u3...,"GELEEN, PAYS-BAS"


In [82]:
df_nike_blazer_Homme['Gender']="Men"

In [83]:
df_nike_blazer_Homme.to_csv("data_NikeBlazer_HOMME.csv",index=False)

##### Enfant

In [86]:
url = "https://www.vinted.fr/catalog?search_text=Nike%20Blazer&brand_ids[]=53&status_ids[]=6&status_ids[]=1&status_ids[]=2&catalog[]=1193"
vinted_data_blazer_enfant = get_vinted_data(url,50)
df_nike_blazer_Enfant= process_dataframe(vinted_data_blazer_enfant)
df_nike_blazer_Enfant

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Chaussure Blazer Nike,10.0,NIKE,BLANC,18,https://www.vinted.fr/items/3878327610-chaussu...,https://images1.vinted.net/t/01_022b1_Ae7tE13E...,FRANCE
1,Nike blazer 25,15.0,NIKE,BLANC,25,https://www.vinted.fr/items/3865323717-nike-bl...,https://images1.vinted.net/t/03_013fa_e4CBKazm...,FRANCE
2,Nike blazer Bébé 17,15.0,NIKE,"BLANC, VERT",17,https://www.vinted.fr/items/3827448678-nike-bl...,https://images1.vinted.net/t/01_001a0_443DtADd...,"JUILLAN, FRANCE"
3,Pull enfant Nike,28.0,NIKE,"NOIR, GRIS",XL,https://www.vinted.fr/items/3860530990-pull-en...,https://images1.vinted.net/t/01_004aa_9jbsB4dh...,FRANCE
4,Nike blazer Mid T38,20.0,NIKE,"BLANC, ROSE",38,https://www.vinted.fr/items/3645479505-nike-bl...,https://images1.vinted.net/t/03_00e8f_G2Ek2fFU...,"EURRE, FRANCE"
5,Veste Nike,2.0,NIKE,"NOIR, GRIS",9 ans / 134 cm,https://www.vinted.fr/items/3770348045-veste-n...,https://images1.vinted.net/t/01_002f4_EBXjX44p...,"TOUL, FRANCE"
6,Zapatillas niño/niña Nike,20.0,NIKE,BEIGE,36,https://www.vinted.fr/items/3869681953-zapatil...,https://images1.vinted.net/t/02_00978_mZgEvr6k...,"ZARAGOZA, ESPAGNE"
7,Baskets blazer low,3.0,NIKE,BLANC,29,https://www.vinted.fr/items/3853744571-baskets...,https://images1.vinted.net/t/03_00862_ZgCjuxpW...,"RAISMES, FRANCE"
8,Sneakers Nike Blazer numero 24,45.0,NIKE,"GRIS, ROSE",24,https://www.vinted.fr/items/3507394386-sneaker...,https://images1.vinted.net/t/02_02155_Su6bQMET...,ITALIE
9,blazer mixte,15.0,NIKE,"NOIR, BLANC",17,https://www.vinted.fr/items/3872682534-blazer-...,https://images1.vinted.net/t/02_01b8b_E6Y4tPRV...,FRANCE


In [87]:
df_nike_blazer_Enfant['Gender']="Kid"

In [88]:
df_nike_blazer_Enfant.to_csv("data_NikeBlazer_ENFANT.csv",index=False)

##### Concataned Dataframe of Nike Blazer

In [89]:
df_nike_blazer_Femme = pd.read_csv("data_NikeBlazer_FEMME.csv",sep=",")
df_nike_blazer_Homme = pd.read_csv("data_NikeBlazer_HOMME.csv",sep=",")
df_nike_blazer_Enfant = pd.read_csv("data_NikeBlazer_ENFANT.csv",sep=",")

In [90]:
df_nike_blazer = pd.concat([df_nike_blazer_Femme, df_nike_blazer_Homme, df_nike_blazer_Enfant], axis=0, ignore_index=True)
df_nike_blazer

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,Nike blazer,18.0,NIKE,"NOIR, BLANC",38.0,https://www.vinted.fr/items/3876910512-nike-bl...,https://images1.vinted.net/t/03_01ead_Qq9rWDqE...,FRANCE,Women
1,Nike Blazer,20.0,NIKE,"NOIR, BLANC",35.0,https://www.vinted.fr/items/3848826530-nike-bl...,https://images1.vinted.net/t/01_008bd_rpQ6BAxe...,"BARCELONA, ESPAGNE",Women
2,Nike blazer,8.0,NIKE,"NOIR, BLANC",38.5,https://www.vinted.fr/items/3221377177-nike-bl...,https://images1.vinted.net/t/01_007ea_Vye88dkb...,"MESSINA, ITALIE",Women
3,Nike blazer,10.0,NIKE,"BLANC, JAUNE",37.5,https://www.vinted.fr/items/3148333128-nike-bl...,https://images1.vinted.net/t/02_01b0c_E4Rk5HB1...,ITALIE,Women
4,Nike Blazer,14.9,NIKE,"BLANC, MARINE",40.0,https://www.vinted.fr/items/3884765098-nike-bl...,https://images1.vinted.net/t/01_002c1_cctsdgjk...,PORTUGAL,Women
...,...,...,...,...,...,...,...,...,...
2079,Nike Blazer Mid’77,20.0,NIKE,BLANC,25,https://www.vinted.fr/items/3625072475-nike-bl...,https://images1.vinted.net/t/03_01c9a_2oDi848M...,FRANCE,Kid
2080,Nike blazer,5.0,NIKE,NOIR,21,https://www.vinted.fr/items/3414036077-nike-bl...,https://images1.vinted.net/t/02_00c76_SdjoBUx3...,"BONCHAMP-LÈS-LAVAL, FRANCE",Kid
2081,Nike Blazer,20.0,NIKE,BLANC,21,https://www.vinted.fr/items/3764712567-nike-bl...,https://images1.vinted.net/t/01_01c21_CgpijUmf...,"DOUVRES-LA-DÉLIVRANDE, FRANCE",Kid
2082,Teddy Nike doublé,8.0,NIKE,MARINE,L,https://www.vinted.fr/items/3828694290-teddy-n...,https://images1.vinted.net/t/01_0125d_v9XFMGon...,FRANCE,Kid


In [91]:
# Adding columns to the DataFrame for the approximate carbon footprint and Nike's fixed value
df_nike_blazer["Approximate Carbon Footprint (kg CO2)"] = df_nike_blazer["Localisation"].apply(calculate_approximate_carbon_footprint)
df_nike_blazer["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] = 405  

# Adding a column for Carbon Profit
df_nike_blazer["Carbon Profit (kg CO2)"] = df_nike_blazer["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] - df_nike_blazer["Approximate Carbon Footprint (kg CO2)"]

df_nike_blazer


,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2)
0,Nike blazer,18.0,NIKE,"NOIR, BLANC",38.0,https://www.vinted.fr/items/3876910512-nike-bl...,https://images1.vinted.net/t/03_01ead_Qq9rWDqE...,FRANCE,Women,50.586158,405,354.413842
1,Nike Blazer,20.0,NIKE,"NOIR, BLANC",35.0,https://www.vinted.fr/items/3848826530-nike-bl...,https://images1.vinted.net/t/01_008bd_rpQ6BAxe...,"BARCELONA, ESPAGNE",Women,166.139391,405,238.860609
2,Nike blazer,8.0,NIKE,"NOIR, BLANC",38.5,https://www.vinted.fr/items/3221377177-nike-bl...,https://images1.vinted.net/t/01_007ea_Vye88dkb...,"MESSINA, ITALIE",Women,317.990079,405,87.009921
3,Nike blazer,10.0,NIKE,"BLANC, JAUNE",37.5,https://www.vinted.fr/items/3148333128-nike-bl...,https://images1.vinted.net/t/02_01b0c_E4Rk5HB1...,ITALIE,Women,211.591248,405,193.408752
4,Nike Blazer,14.9,NIKE,"BLANC, MARINE",40.0,https://www.vinted.fr/items/3884765098-nike-bl...,https://images1.vinted.net/t/01_002c1_cctsdgjk...,PORTUGAL,Women,263.698433,405,141.301567
...,...,...,...,...,...,...,...,...,...,...,...,...
2079,Nike Blazer Mid’77,20.0,NIKE,BLANC,25,https://www.vinted.fr/items/3625072475-nike-bl...,https://images1.vinted.net/t/03_01c9a_2oDi848M...,FRANCE,Kid,50.586158,405,354.413842
2080,Nike blazer,5.0,NIKE,NOIR,21,https://www.vinted.fr/items/3414036077-nike-bl...,https://images1.vinted.net/t/02_00c76_SdjoBUx3...,"BONCHAMP-LÈS-LAVAL, FRANCE",Kid,48.360988,405,356.639012
2081,Nike Blazer,20.0,NIKE,BLANC,21,https://www.vinted.fr/items/3764712567-nike-bl...,https://images1.vinted.net/t/01_01c21_CgpijUmf...,"DOUVRES-LA-DÉLIVRANDE, FRANCE",Kid,41.036671,405,363.963329
2082,Teddy Nike doublé,8.0,NIKE,MARINE,L,https://www.vinted.fr/items/3828694290-teddy-n...,https://images1.vinted.net/t/01_0125d_v9XFMGon...,FRANCE,Kid,50.586158,405,354.413842


In [93]:
url2 = "https://www.nike.com/fr/w?q=nike%20blazer&vst=nike%20blazer"
average_nike_price2=get_nike_prices(url2)
average_nike_price2

88.6856756756756

In [94]:
df_nike_blazer.sort_values(by='Price', inplace=True)

# Calculate profit made for Vinted data
df_nike_blazer['Average price of the product on Nike'] = average_nike_price2
df_nike_blazer['Profit Made'] = df_nike_blazer.apply(lambda row: calculate_profit_made(row, average_nike_price2), axis=1)

df_nike_blazer

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
2039,Veste Nike,2.0,NIKE,"NOIR, GRIS",9 ans / 134 cm,https://www.vinted.fr/items/3770348045-veste-n...,https://images1.vinted.net/t/01_002f4_EBXjX44p...,"TOUL, FRANCE",Kid,52.225935,405,352.774065,88.685676,86.685676
2074,Veste Nike 12-18mois,2.0,NIKE,BLEU,12-18 mois / 80 cm,https://www.vinted.fr/items/3785072417-veste-n...,https://images1.vinted.net/t/03_00566_yUZkgFRS...,FRANCE,Kid,50.586158,405,354.413842,88.685676,86.685676
2056,Veste chaude fille nike 4 ans,2.0,NIKE,BEIGE,4 ans / 104 cm,https://www.vinted.fr/items/3709073037-veste-c...,https://images1.vinted.net/t/01_01b71_ZkUEQPi9...,"MOURMELON-LE-GRAND, FRANCE",Kid,30.126044,405,374.873956,88.685676,86.685676
2045,Veste Nike rouge,3.0,NIKE,ROUGE,12 ans / 152 cm,https://www.vinted.fr/items/3155260497-veste-n...,https://images1.vinted.net/t/02_01b9b_xkmFsgFN...,"GONESSE, FRANCE",Kid,3.222731,405,401.777269,88.685676,85.685676
2041,Baskets blazer low,3.0,NIKE,BLANC,29,https://www.vinted.fr/items/3853744571-baskets...,https://images1.vinted.net/t/03_00862_ZgCjuxpW...,"RAISMES, FRANCE",Kid,37.849440,405,367.150560,88.685676,85.685676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,Nike SB Blazer Mid Iso,130.0,NIKE,"NOIR, ROUGE",45,https://www.vinted.fr/items/3755007513-nike-sb...,https://images1.vinted.net/t/01_008e4_CMrsm7vQ...,"ORLÉANS, FRANCE",Men,22.208110,405,382.791890,88.685676,-41.314324
1978,Nike blazer,130.0,NIKE,NOIR,"42,5",https://www.vinted.fr/items/751665575-nike-bla...,https://images1.vinted.net/t/02_00915_bvNnH3RW...,"CHAURAY, FRANCE",Men,68.994685,405,336.005315,88.685676,-41.314324
571,Nike Blazer Low Sacai ‘Patent White’ EU 40,150.0,NIKE,"NOIR, BLANC",40.0,https://www.vinted.fr/items/3886271027-nike-bl...,https://images1.vinted.net/t/03_01fc1_A33gzH7i...,"AMSTERDAM, PAYS-BAS",Women,86.089577,405,318.910423,88.685676,-61.314324
1457,Nike Blazer,160.0,NIKE,"NOIR, ORANGE",44,https://www.vinted.fr/items/3735160628-nike-bl...,https://images1.vinted.net/t/03_00fd1_FfafcHkU...,"ANNECY, FRANCE",Men,87.029014,405,317.970986,88.685676,-71.314324


In [95]:
df_nike_blazer= df_nike_blazer.dropna(subset=['Profit Made'])
df_nike_blazer

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
2039,Veste Nike,2.0,NIKE,"NOIR, GRIS",9 ans / 134 cm,https://www.vinted.fr/items/3770348045-veste-n...,https://images1.vinted.net/t/01_002f4_EBXjX44p...,"TOUL, FRANCE",Kid,52.225935,405,352.774065,88.685676,86.685676
2074,Veste Nike 12-18mois,2.0,NIKE,BLEU,12-18 mois / 80 cm,https://www.vinted.fr/items/3785072417-veste-n...,https://images1.vinted.net/t/03_00566_yUZkgFRS...,FRANCE,Kid,50.586158,405,354.413842,88.685676,86.685676
2056,Veste chaude fille nike 4 ans,2.0,NIKE,BEIGE,4 ans / 104 cm,https://www.vinted.fr/items/3709073037-veste-c...,https://images1.vinted.net/t/01_01b71_ZkUEQPi9...,"MOURMELON-LE-GRAND, FRANCE",Kid,30.126044,405,374.873956,88.685676,86.685676
2045,Veste Nike rouge,3.0,NIKE,ROUGE,12 ans / 152 cm,https://www.vinted.fr/items/3155260497-veste-n...,https://images1.vinted.net/t/02_01b9b_xkmFsgFN...,"GONESSE, FRANCE",Kid,3.222731,405,401.777269,88.685676,85.685676
2041,Baskets blazer low,3.0,NIKE,BLANC,29,https://www.vinted.fr/items/3853744571-baskets...,https://images1.vinted.net/t/03_00862_ZgCjuxpW...,"RAISMES, FRANCE",Kid,37.849440,405,367.150560,88.685676,85.685676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144,Nike Blazer mid 77 sketch,125.0,NIKE,BLANC,42,https://www.vinted.fr/items/3882066403-nike-bl...,https://images1.vinted.net/t/02_0189f_2u6pq2e6...,"AULNOY-LEZ-VALENCIENNES, FRANCE",Men,36.960615,405,368.039385,88.685676,-36.314324
1464,Nike SB Blazer Mid Iso,130.0,NIKE,"NOIR, ROUGE",45,https://www.vinted.fr/items/3755007513-nike-sb...,https://images1.vinted.net/t/01_008e4_CMrsm7vQ...,"ORLÉANS, FRANCE",Men,22.208110,405,382.791890,88.685676,-41.314324
1978,Nike blazer,130.0,NIKE,NOIR,"42,5",https://www.vinted.fr/items/751665575-nike-bla...,https://images1.vinted.net/t/02_00915_bvNnH3RW...,"CHAURAY, FRANCE",Men,68.994685,405,336.005315,88.685676,-41.314324
571,Nike Blazer Low Sacai ‘Patent White’ EU 40,150.0,NIKE,"NOIR, BLANC",40.0,https://www.vinted.fr/items/3886271027-nike-bl...,https://images1.vinted.net/t/03_01fc1_A33gzH7i...,"AMSTERDAM, PAYS-BAS",Women,86.089577,405,318.910423,88.685676,-61.314324


In [96]:
df_nike_blazer.to_csv("AllData_NikeBlazer.csv",index=False)

### Nike Dunk

##### Femme

In [97]:
url = "https://www.vinted.fr/catalog?search_text=Nike%20Dunk&brand_ids[]=53&status_ids[]=6&status_ids[]=1&status_ids[]=2&catalog[]=16"
vinted_data_dunk_femme = get_vinted_data(url,1017)
df_nike_dunk_Femme = process_dataframe(vinted_data_dunk_femme)
df_nike_dunk_Femme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Dunk Low (Apenas um sapato),25.0,NIKE,"BLANC, BEIGE",42.5,https://www.vinted.fr/items/3883478564-dunk-lo...,https://images1.vinted.net/t/03_01bb9_U1DAbfqN...,PORTUGAL
1,Dunk low mauve,50.0,NIKE,"BLANC, LILA",40,https://www.vinted.fr/items/3880336776-dunk-lo...,https://images1.vinted.net/t/03_01808_FSAfuEfw...,FRANCE
2,Dunk Low Black White Panda sneakers 39,50.0,NIKE,"NOIR, BLANC",39,https://www.vinted.fr/items/3888062557-dunk-lo...,https://images1.vinted.net/t/01_00e4d_Z7QVyRnZ...,FRANCE
3,Nike Dunk Low Triple Pink (GS) 38.5,53.1,NIKE,"BLANC, ROSE",38.5,https://www.vinted.fr/items/3869761967-nike-du...,https://images1.vinted.net/t/02_0063b_KeXYyRom...,FRANCE
4,Nike dunk,50.0,NIKE,KAKI,37.5,https://www.vinted.fr/items/3882268289-nike-du...,https://images1.vinted.net/t/01_0113b_dDci1qQb...,"L'ÎLE-D'OLONNE, FRANCE"
...,...,...,...,...,...,...,...,...
1012,Dunk low,75.0,NIKE,"NOIR, GRIS",39,https://www.vinted.fr/items/3880761992-dunk-lo...,https://images1.vinted.net/t/02_013ac_yedRFxA8...,"ANNEMASSE, FRANCE"
1013,Nike dunk panda alte taglia 38 usate ma in ott...,50.0,NIKE,"NOIR, BLANC",38,https://www.vinted.fr/items/3889109666-nike-du...,https://images1.vinted.net/t/03_01e0e_faqQqSJK...,"FRASCATI, ITALIE"
1014,Scarpe Nike Dunk Low Chicago taglia 36 Unisex,85.0,NIKE,ROUGE,36,https://www.vinted.fr/items/3890493882-scarpe-...,https://images1.vinted.net/t/02_004b2_WJzQFBWh...,"ROMA, ITALIE"
1015,Dunk Low Rose Barbie,65.0,NIKE,"ROSE, ROSE",38,https://www.vinted.fr/items/3878118271-dunk-lo...,https://images1.vinted.net/t/03_00221_1uckF5T9...,"CHÂTEAUROUX, FRANCE"


In [98]:
df_nike_dunk_Femme['Gender']="Women"

In [116]:
df_nike_dunk_Femme.to_csv("data_NikeDunk_FEMME.csv",index=False)

##### Homme

In [100]:
url = "https://www.vinted.fr/catalog?search_text=Nike%20Dunk&brand_ids[]=53&status_ids[]=6&status_ids[]=1&status_ids[]=2&catalog[]=5"
vinted_data_dunk_homme = get_vinted_data(url,1017)
df_nike_dunk_Homme = process_dataframe(vinted_data_dunk_homme)
df_nike_dunk_Homme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Dunk low,1.0,NIKE,"BLANC, BLEU","44,5",https://www.vinted.fr/items/3890663184-dunk-lo...,https://images1.vinted.net/t/03_0186f_nVyJSyue...,"GIF-SUR-YVETTE, FRANCE"
1,Nike dunk low medium curry,15.0,NIKE,"BEIGE, MARRON",42,https://www.vinted.fr/items/3891813762-nike-du...,https://images1.vinted.net/t/03_02165_aQWPKUpp...,"LA RONDE, FRANCE"
2,Nike dunk,60.0,NIKE,"BLEU, MARINE",42,https://www.vinted.fr/items/3889531447-nike-du...,https://images1.vinted.net/t/03_02206_9jZdtkqK...,"BURJASSOT, ESPAGNE"
3,Nike dunk high Varsity purple 2010,21.0,NIKE,"BLANC, VIOLET",42,https://www.vinted.fr/items/3890405989-nike-du...,https://images1.vinted.net/t/02_01b88_PjUXazfm...,FRANCE
4,dunk low meidum olive 46,55.0,NIKE,"BLANC, KAKI",46,https://www.vinted.fr/items/3889828469-dunk-lo...,https://images1.vinted.net/t/02_006c1_PBwGg8U2...,"VILLABÉ, FRANCE"
...,...,...,...,...,...,...,...,...
1012,Nike dunk low peach cream,70.0,NIKE,"BLANC, ORANGE","40,5",https://www.vinted.fr/items/3888379933-nike-du...,https://images1.vinted.net/t/01_00f71_zs95CEKw...,FRANCE
1013,Nike dunk 43 excellent état,80.0,NIKE,"NOIR, BLEU",43,https://www.vinted.fr/items/3887753953-nike-du...,https://images1.vinted.net/t/03_00708_JLWkPok1...,"VILLENAVE-D'ORNON, FRANCE"
1014,Nike SB dunk low powerpuff pink,330.0,NIKE,"ORANGE, ROSE","42,5",https://www.vinted.fr/items/3885876747-nike-sb...,https://images1.vinted.net/t/01_01cd3_hSqCUA92...,"MARCQ-EN-BARŒUL, FRANCE"
1015,Nike zoomx invincible 3 - Running course fitne...,129.0,NIKE,"BLANC, VIOLET",41,https://www.vinted.fr/items/3891163578-nike-zo...,https://images1.vinted.net/t/01_017b8_872Zyamo...,"ALICANTE/ALACANT, ESPAGNE"


In [101]:
df_nike_dunk_Homme['Gender']="Men"

In [115]:
df_nike_dunk_Homme.to_csv("data_NikeDunk_HOMME.csv",index=False)

##### Enfant

In [103]:
url = "https://www.vinted.fr/catalog?search_text=Nike%20Dunk&brand_ids[]=53&status_ids[]=6&status_ids[]=1&status_ids[]=2&catalog[]=1193"
vinted_data_dunk_enfant = get_vinted_data(url,100)
df_nike_dunk_Enfant = process_dataframe(vinted_data_dunk_enfant)
df_nike_dunk_Enfant

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Nike dunk bébé grise taille 25,15.00,NIKE,GRIS,25,https://www.vinted.fr/items/3888601040-nike-du...,https://images1.vinted.net/t/01_015e7_CeyX15wo...,"SAINT-AVOLD, FRANCE"
1,Dunk Nike enfant,25.00,NIKE,"NOIR, BLANC",22,https://www.vinted.fr/items/3883877271-dunk-ni...,https://images1.vinted.net/t/03_026a5_y3xvsoki...,"REMERSDAAL, BELGIQUE"
2,Nike Dunk Low Black and whte panda 38,41.00,NIKE,"NOIR, BLANC",38,https://www.vinted.fr/items/3881795207-nike-du...,https://images1.vinted.net/t/02_020ce_tnvqQAuG...,PAYS-BAS
3,Nike Dunk Flamingo sneakers EU 38,51.32,NIKE,GRIS,38,https://www.vinted.fr/items/3874526946-nike-du...,https://images1.vinted.net/t/03_01020_pfAenhWB...,FRANCE
4,Nike dunk panda,20.00,NIKE,NOIR,22,https://www.vinted.fr/items/3891578411-nike-du...,https://images1.vinted.net/t/02_01ec4_VPynArmn...,"PAVILLY, FRANCE"
...,...,...,...,...,...,...,...,...
95,Nike Dunk Grey White taille 38,52.36,NIKE,GRIS,38,https://www.vinted.fr/items/3888579362-nike-du...,https://images1.vinted.net/t/02_003dd_Zw67U79F...,"SENS, FRANCE"
96,Nike dunk panda,35.00,NIKE,"NOIR, BLANC",19,https://www.vinted.fr/items/3886580820-nike-du...,https://images1.vinted.net/t/01_00fda_P3G1rLEf...,"ISTRES, FRANCE"
97,Basket NIKE dunk low,35.00,NIKE,ROSE,33,https://www.vinted.fr/items/3888617873-basket-...,https://images1.vinted.net/t/02_01763_pYdAocoL...,"MARSEILLE 08, FRANCE"
98,Nike dunk high. 36,60.00,NIKE,MULTICOLORE,36,https://www.vinted.fr/items/3876733400-nike-du...,https://images1.vinted.net/t/03_02280_ohB1Vgzf...,"ARGELÈS-SUR-MER, FRANCE"


In [104]:
df_nike_dunk_Enfant['Gender']="Kid"

In [114]:
df_nike_dunk_Enfant.to_csv("data_NikeDunk_ENFANT.csv",index=False)

##### Concataned Dataframe of Nike Blazer

In [117]:
df_nike_dunk_Femme = pd.read_csv("data_NikeDunk_FEMME.csv",sep=",")
df_nike_dunk_Homme = pd.read_csv("data_NikeDunk_HOMME.csv",sep=",")
df_nike_dunk_Enfant = pd.read_csv("data_NikeDunk_ENFANT.csv",sep=",")

In [118]:
df_nike_dunk = pd.concat([df_nike_dunk_Femme, df_nike_dunk_Homme, df_nike_dunk_Enfant], axis=0, ignore_index=True)
df_nike_dunk

,Unnamed: 0,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,0,Dunk Low (Apenas um sapato),25.00,NIKE,"BLANC, BEIGE",42.5,https://www.vinted.fr/items/3883478564-dunk-lo...,https://images1.vinted.net/t/03_01bb9_U1DAbfqN...,PORTUGAL,Women
1,1,Dunk low mauve,50.00,NIKE,"BLANC, LILA",40,https://www.vinted.fr/items/3880336776-dunk-lo...,https://images1.vinted.net/t/03_01808_FSAfuEfw...,FRANCE,Women
2,2,Dunk Low Black White Panda sneakers 39,50.00,NIKE,"NOIR, BLANC",39,https://www.vinted.fr/items/3888062557-dunk-lo...,https://images1.vinted.net/t/01_00e4d_Z7QVyRnZ...,FRANCE,Women
3,3,Nike Dunk Low Triple Pink (GS) 38.5,53.10,NIKE,"BLANC, ROSE",38.5,https://www.vinted.fr/items/3869761967-nike-du...,https://images1.vinted.net/t/02_0063b_KeXYyRom...,FRANCE,Women
4,4,Nike dunk,50.00,NIKE,KAKI,37.5,https://www.vinted.fr/items/3882268289-nike-du...,https://images1.vinted.net/t/01_0113b_dDci1qQb...,"L'ÎLE-D'OLONNE, FRANCE",Women
...,...,...,...,...,...,...,...,...,...,...
2129,95,Nike Dunk Grey White taille 38,52.36,NIKE,GRIS,38,https://www.vinted.fr/items/3888579362-nike-du...,https://images1.vinted.net/t/02_003dd_Zw67U79F...,"SENS, FRANCE",Kid
2130,96,Nike dunk panda,35.00,NIKE,"NOIR, BLANC",19,https://www.vinted.fr/items/3886580820-nike-du...,https://images1.vinted.net/t/01_00fda_P3G1rLEf...,"ISTRES, FRANCE",Kid
2131,97,Basket NIKE dunk low,35.00,NIKE,ROSE,33,https://www.vinted.fr/items/3888617873-basket-...,https://images1.vinted.net/t/02_01763_pYdAocoL...,"MARSEILLE 08, FRANCE",Kid
2132,98,Nike dunk high. 36,60.00,NIKE,MULTICOLORE,36,https://www.vinted.fr/items/3876733400-nike-du...,https://images1.vinted.net/t/03_02280_ohB1Vgzf...,"ARGELÈS-SUR-MER, FRANCE",Kid


In [108]:
# Adding columns to the DataFrame for the approximate carbon footprint and Nike's fixed value
df_nike_dunk["Approximate Carbon Footprint (kg CO2)"] = df_nike_dunk["Localisation"].apply(calculate_approximate_carbon_footprint)
df_nike_dunk["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] = 405  

# Adding a column for Carbon Profit
df_nike_dunk["Carbon Profit (kg CO2)"] = df_nike_dunk["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] - df_nike_dunk["Approximate Carbon Footprint (kg CO2)"]

df_nike_dunk


,Unnamed: 0,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2)
0,0,Dunk Low (Apenas um sapato),25.00,NIKE,"BLANC, BEIGE",42.5,https://www.vinted.fr/items/3883478564-dunk-lo...,https://images1.vinted.net/t/03_01bb9_U1DAbfqN...,PORTUGAL,Women,263.698433,405,141.301567
1,1,Dunk low mauve,50.00,NIKE,"BLANC, LILA",40,https://www.vinted.fr/items/3880336776-dunk-lo...,https://images1.vinted.net/t/03_01808_FSAfuEfw...,FRANCE,Women,50.586158,405,354.413842
2,2,Dunk Low Black White Panda sneakers 39,50.00,NIKE,"NOIR, BLANC",39,https://www.vinted.fr/items/3888062557-dunk-lo...,https://images1.vinted.net/t/01_00e4d_Z7QVyRnZ...,FRANCE,Women,50.586158,405,354.413842
3,3,Nike Dunk Low Triple Pink (GS) 38.5,53.10,NIKE,"BLANC, ROSE",38.5,https://www.vinted.fr/items/3869761967-nike-du...,https://images1.vinted.net/t/02_0063b_KeXYyRom...,FRANCE,Women,50.586158,405,354.413842
4,4,Nike dunk,50.00,NIKE,KAKI,37.5,https://www.vinted.fr/items/3882268289-nike-du...,https://images1.vinted.net/t/01_0113b_dDci1qQb...,"L'ÎLE-D'OLONNE, FRANCE",Women,80.325189,405,324.674811
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2129,95,Nike Dunk Grey White taille 38,52.36,NIKE,GRIS,38,https://www.vinted.fr/items/3888579362-nike-du...,https://images1.vinted.net/t/02_003dd_Zw67U79F...,"SENS, FRANCE",Kid,20.088099,405,384.911901
2130,96,Nike dunk panda,35.00,NIKE,"NOIR, BLANC",19,https://www.vinted.fr/items/3886580820-nike-du...,https://images1.vinted.net/t/01_00fda_P3G1rLEf...,"ISTRES, FRANCE",Kid,125.534017,405,279.465983
2131,97,Basket NIKE dunk low,35.00,NIKE,ROSE,33,https://www.vinted.fr/items/3888617873-basket-...,https://images1.vinted.net/t/02_01763_pYdAocoL...,"MARSEILLE 08, FRANCE",Kid,133.267433,405,271.732567
2132,98,Nike dunk high. 36,60.00,NIKE,MULTICOLORE,36,https://www.vinted.fr/items/3876733400-nike-du...,https://images1.vinted.net/t/03_02280_ohB1Vgzf...,"ARGELÈS-SUR-MER, FRANCE",Kid,140.631484,405,264.368516


In [110]:
url3 = "https://www.nike.com/fr/w?q=nike%20dunk&vst=nike%20dunk"
average_nike_price3=get_nike_prices(url3)
average_nike_price3

115.88870967741933

In [111]:
df_nike_dunk.sort_values(by='Price', inplace=True)

# Calculate profit made for Vinted data
df_nike_dunk['Average price of the product on Nike'] = average_nike_price3
df_nike_dunk['Profit Made'] = df_nike_dunk.apply(lambda row: calculate_profit_made(row, average_nike_price3), axis=1)

df_nike_dunk

,Unnamed: 0,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
1017,0,Dunk low,1.0,NIKE,"BLANC, BLEU","44,5",https://www.vinted.fr/items/3890663184-dunk-lo...,https://images1.vinted.net/t/03_0186f_nVyJSyue...,"GIF-SUR-YVETTE, FRANCE",Men,4.698729,405,400.301271,115.88871,114.88871
2085,51,Stikers autocollants Nike Dunk low Off-White e...,3.5,NIKE,MULTICOLORE,Taille non spécifiée,https://www.vinted.fr/items/3832550323-stikers...,https://images1.vinted.net/t/02_003d4_6uf2Pfz6...,FRANCE,Kid,50.586158,405,354.413842,115.88871,112.38871
49,49,dunk low,5.0,NIKE,"BLANC, ROUGE",36.5,https://www.vinted.fr/items/3775343147-dunk-lo...,https://images1.vinted.net/t/01_01d04_8P1JeNMQ...,"BUSIGNY, FRANCE",Women,30.815128,405,374.184872,115.88871,110.88871
2066,32,Nike dunk,9.0,NIKE,"NOIR, BLANC",24,https://www.vinted.fr/items/3798792190-nike-du...,https://images1.vinted.net/t/01_01adb_QX91JnAe...,"ÉPINAY-SUR-SEINE, FRANCE",Kid,2.203805,405,402.796195,115.88871,106.88871
2052,18,Nike dunk bambino/a,10.0,NIKE,"NOIR, BLANC",28,https://www.vinted.fr/items/3884037167-nike-du...,https://images1.vinted.net/t/02_00e82_hYs9Zuoc...,"TORINO, ITALIE",Kid,116.904387,405,288.095613,115.88871,105.88871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,971,Dunk vert d’eau,NaN,"110,00 €",MENTHE,Nike,https://www.vinted.fr/items/3884343389-dunk-ve...,https://images1.vinted.net/t/01_02563_G5TvpHdH...,"MESSIGNY-ET-VANTOUX, FRANCE",Women,51.070908,405,353.929092,115.88871,NaN
1324,307,Nike dunk low black,NaN,NaN,"NOIR, MULTICOLORE","99,99 €",https://www.vinted.fr/items/3889188498-nike-du...,https://images1.vinted.net/t/02_01c0a_A66J5a3j...,"BIANCAVILLA, ITALIE",Men,320.565831,405,84.434169,115.88871,NaN
1432,415,Nike Dunk Low triple White,NaN,"60,00 €",BLANC,Nike,https://www.vinted.fr/items/3889954266-nike-du...,https://images1.vinted.net/t/03_00ffb_Ft2h4pec...,"VILLEURBANNE, FRANCE",Men,78.532560,405,326.467440,115.88871,NaN
1870,853,Nike Dunk low marron et blanche,NaN,"119,90 €","BLANC, MARRON",Nike,https://www.vinted.fr/items/3890567523-nike-du...,https://images1.vinted.net/t/03_00355_6wGHwnvg...,FRANCE,Men,50.586158,405,354.413842,115.88871,NaN


In [112]:
df_nike_dunk= df_nike_dunk.dropna(subset=['Profit Made'])
df_nike_dunk

,Unnamed: 0,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
1017,0,Dunk low,1.0,NIKE,"BLANC, BLEU","44,5",https://www.vinted.fr/items/3890663184-dunk-lo...,https://images1.vinted.net/t/03_0186f_nVyJSyue...,"GIF-SUR-YVETTE, FRANCE",Men,4.698729,405,400.301271,115.88871,114.88871
2085,51,Stikers autocollants Nike Dunk low Off-White e...,3.5,NIKE,MULTICOLORE,Taille non spécifiée,https://www.vinted.fr/items/3832550323-stikers...,https://images1.vinted.net/t/02_003d4_6uf2Pfz6...,FRANCE,Kid,50.586158,405,354.413842,115.88871,112.38871
49,49,dunk low,5.0,NIKE,"BLANC, ROUGE",36.5,https://www.vinted.fr/items/3775343147-dunk-lo...,https://images1.vinted.net/t/01_01d04_8P1JeNMQ...,"BUSIGNY, FRANCE",Women,30.815128,405,374.184872,115.88871,110.88871
2066,32,Nike dunk,9.0,NIKE,"NOIR, BLANC",24,https://www.vinted.fr/items/3798792190-nike-du...,https://images1.vinted.net/t/01_01adb_QX91JnAe...,"ÉPINAY-SUR-SEINE, FRANCE",Kid,2.203805,405,402.796195,115.88871,106.88871
2052,18,Nike dunk bambino/a,10.0,NIKE,"NOIR, BLANC",28,https://www.vinted.fr/items/3884037167-nike-du...,https://images1.vinted.net/t/02_00e82_hYs9Zuoc...,"TORINO, ITALIE",Kid,116.904387,405,288.095613,115.88871,105.88871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,332,Nike Dunk Sb Born × Raised,585.0,NIKE,"BLANC, BLEU",46,https://www.vinted.fr/items/3891293225-nike-du...,https://images1.vinted.net/t/02_00874_RdU8S1Ts...,"GUIPAVAS, FRANCE",Men,99.935728,405,305.064272,115.88871,-469.11129
1887,870,Nike Dunk High Supreme Red 2007,650.0,NIKE,"NOIR, ROUGE",41,https://www.vinted.fr/items/3890565130-nike-du...,https://images1.vinted.net/t/02_02491_TXJG8VBj...,FRANCE,Men,50.586158,405,354.413842,115.88871,-534.11129
1171,154,Nike Dunk Sb powerpuff girls 10us pack,820.0,NIKE,"BLEU, VERT",44,https://www.vinted.fr/items/3886466699-nike-du...,https://images1.vinted.net/t/03_016de_KDjW3sES...,"BORDEAUX, FRANCE",Men,99.834891,405,305.165109,115.88871,-704.11129
1364,347,Pack Nike Dunk Low Sb Powerpuff,900.0,NIKE,"ROSE, BLEU",41,https://www.vinted.fr/items/3891663889-pack-ni...,https://images1.vinted.net/t/01_01b3a_h4jaVJkG...,FRANCE,Men,50.586158,405,354.413842,115.88871,-784.11129


In [113]:
df_nike_dunk.to_csv("AllData_NikeDunk.csv",index=False)

### Nike Mag

##### Femme

In [7]:
url = "https://www.vinted.fr/catalog?search_text=Nike%20Mag&brand_ids[]=53&status_ids[]=6&status_ids[]=1&status_ids[]=2&catalog[]=16"
vinted_data_mag_femme = get_vinted_data(url,500)
df_nike_mag_Femme = process_dataframe(vinted_data_mag_femme)
df_nike_mag_Femme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Nike Blazer,7.5,NIKE,"BLANC, CORAIL",38.5,https://www.vinted.fr/items/3808211046-nike-bl...,https://images1.vinted.net/t/02_01ee5_jJaEXHni...,"ROTTERDAM, PAYS-BAS"
1,Sneakers - Nike Roshe One Valentine's Day,10.0,NIKE,"BLANC, ROUGE",37.5,https://www.vinted.fr/items/3883913822-sneaker...,https://images1.vinted.net/t/02_0130f_JDdNTYHx...,PAYS-BAS
2,Prachtige Air force 1 by Nike,20.0,NIKE,"BLANC, MULTICOLORE",38.5,https://www.vinted.fr/items/3859318004-prachti...,https://images1.vinted.net/t/03_00385_43gpofFH...,PAYS-BAS
3,Nike sneakers,30.0,NIKE,"BLANC, BEIGE",38.5,https://www.vinted.fr/items/3894770172-nike-sn...,https://images1.vinted.net/t/02_00c06_X6wpDDpS...,BELGIQUE
4,Nike sportschoenen,20.0,NIKE,"BLANC, CRÈME",37.5,https://www.vinted.fr/items/3894774215-nike-sp...,https://images1.vinted.net/t/01_01b79_yhUBrBWD...,BELGIQUE
...,...,...,...,...,...,...,...,...
495,"Nike air max 97 Silver maat 35,5",40.0,NIKE,"GRIS, ARGENTÉ",35.5,https://www.vinted.fr/items/1426695491-nike-ai...,https://images1.vinted.net/t/01_01d06_zayux6BE...,PAYS-BAS
496,Nike airmax 97 purple,90.0,NIKE,"ROSE, LILA",40.5,https://www.vinted.fr/items/3343379795-nike-ai...,https://images1.vinted.net/t/01_004cf_g4AZGZKB...,PAYS-BAS
497,Nike vapormax beige,85.0,NIKE,"BEIGE, CRÈME",38,https://www.vinted.fr/items/3005687251-nike-va...,https://images1.vinted.net/t/02_00eee_X1NZTYhe...,"MIDDELBURG, PAYS-BAS"
498,Nike court air zoom gp turbo,50.0,NIKE,BLANC,35.5,https://www.vinted.fr/items/1251028451-nike-co...,https://images1.vinted.net/t/03_0158a_yzuM5muH...,PAYS-BAS


In [8]:
df_nike_mag_Femme['Gender']="Women"

In [9]:
df_nike_mag_Femme.to_csv("data_NikeMag_FEMME.csv",index=False)

##### Homme

In [9]:
url = "https://www.vinted.fr/catalog?search_text=Nike%20Mag&brand_ids[]=53&status_ids[]=6&status_ids[]=1&status_ids[]=2&catalog[]=1231"
vinted_data_mag_homme = get_vinted_data(url,300)
df_nike_mag_Homme = process_dataframe(vinted_data_mag_homme)
df_nike_mag_Homme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Nike court vision mid nieuw! Unisex,53.00,NIKE,NOIR,40,https://www.vinted.fr/items/3879596091-nike-co...,https://images1.vinted.net/t/03_0199d_zPk7taTN...,PAYS-BAS
1,Zwarte Nike Air Max schoenen,19.99,NIKE,NOIR,41,https://www.vinted.fr/items/3911558584-zwarte-...,https://images1.vinted.net/t/03_0181f_wchGzSPb...,"SASSENHEIM, PAYS-BAS"
2,Nike Air Force 1 shadow - zwart maat 39,60.00,NIKE,NOIR,39,https://www.vinted.fr/items/3817934940-nike-ai...,https://images1.vinted.net/t/01_00463_1y4KrFRj...,"HOORN, PAYS-BAS"
3,Nike phantom,100.00,NIKE,"NOIR, ORANGE","45,5",https://www.vinted.fr/items/3856587063-nike-ph...,https://images1.vinted.net/t/02_00843_6fAbBry6...,"EMMEN, PAYS-BAS"
4,Nike Phantom GX Elite,150.00,NIKE,"NOIR, ROSE",44,https://www.vinted.fr/items/3910971158-nike-ph...,https://images1.vinted.net/t/01_023e9_VQjGmKJJ...,PAYS-BAS
...,...,...,...,...,...,...,...,...
295,nike schoenen,45.00,NIKE,"NOIR, GRIS","42,5",https://www.vinted.fr/items/1774868447-nike-sc...,https://images1.vinted.net/t/03_0091d_rd6DYwNP...,PAYS-BAS
296,Nike Swoosh Hunter beige,95.00,NIKE,"BEIGE, ABRICOT",44,https://www.vinted.fr/items/1407804379-nike-sw...,https://images1.vinted.net/t/03_01fce_amhUeZ5V...,FRANCE
297,Nike tear away maat 42.5,100.00,NIKE,MULTICOLORE,"42,5",https://www.vinted.fr/items/2631554089-nike-te...,https://images1.vinted.net/t/02_019da_rAfuwjkq...,"AMSTERDAM, PAYS-BAS"
298,Nike air max 97,50.00,NIKE,ARGENTÉ,38,https://www.vinted.fr/items/276452825-nike-air...,https://images1.vinted.net/t/01_0259e_FYhJvqsz...,PAYS-BAS


In [10]:
df_nike_mag_Homme['Gender']="Men"

In [11]:
df_nike_mag_Homme.to_csv("data_NikeMag_HOMME.csv",index=False)

##### Enfant

In [12]:
url = "https://www.vinted.fr/catalog?search_text=nike%20mag%20chaussures&catalog[]=1193&brand_ids[]=53&status_ids[]=6&status_ids[]=1&status_ids[]=2"
vinted_data_mag_enfant = get_vinted_data(url,80)
df_nike_mag_Enfant = process_dataframe(vinted_data_mag_enfant)
df_nike_mag_Enfant

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Air Force 1,20.00,NIKE,BLANC,21,https://www.vinted.fr/items/3913789611-air-for...,https://images1.vinted.net/t/03_01bdd_iJLktqoA...,"ANTWERP, BELGIQUE"
1,Jongens schoenen Nike,15.00,NIKE,"NOIR, BLANC",28,https://www.vinted.fr/items/3897920907-jongens...,https://images1.vinted.net/t/03_0193f_ZfQk8fGW...,"EINDHOVEN, PAYS-BAS"
2,NIKE Court Borough,20.00,NIKE,"NOIR, ROUGE",21,https://www.vinted.fr/items/3913803200-nike-co...,https://images1.vinted.net/t/02_0168f_hXrudXTx...,"ANTWERP, BELGIQUE"
3,Handige Nike slippers met klittenband maat 36,5.95,NIKE,"BLEU, VERT",36,https://www.vinted.fr/items/3861100337-handige...,https://images1.vinted.net/t/03_013a4_z2Fyyqda...,"SPAARNDAM, PAYS-BAS"
4,"Nike jongens sneakers maat 23,5",10.00,NIKE,"NOIR, ROUGE",23,https://www.vinted.fr/items/3824820965-nike-jo...,https://images1.vinted.net/t/02_00a5d_jW1WMHZd...,"SPIJKENISSE, PAYS-BAS"
...,...,...,...,...,...,...,...,...
75,Nike schoenen,30.00,NIKE,"NOIR, BLANC",36,https://www.vinted.fr/items/1236922541-nike-sc...,https://images1.vinted.net/t/03_01742_NN736grX...,"SILVOLDE, PAYS-BAS"
76,"Nike air max 19,5",20.00,NIKE,"BLEU, MARINE",19,https://www.vinted.fr/items/2171498196-nike-ai...,https://images1.vinted.net/t/03_00e8e_Kz6UUsCV...,PAYS-BAS
77,Nike mt 25,15.00,NIKE,MULTICOLORE,25,https://www.vinted.fr/items/328396909-nike-mt-...,https://images1.vinted.net/t/01_005a8_pnV9Qp3J...,"ROTTERDAM, PAYS-BAS"
78,Nike city of dreams maat 18.5,75.00,NIKE,MULTICOLORE,18,https://www.vinted.fr/items/3408759047-nike-ci...,https://images1.vinted.net/t/01_0169c_ryiDKJkd...,"DE MEERN, PAYS-BAS"


In [13]:
df_nike_mag_Enfant['Gender']="Kid"

In [14]:
df_nike_mag_Enfant.to_csv("data_NikeMag_ENFANT.csv",index=False)

In [16]:

df_nike_mag_Femme = pd.read_csv("data_NikeMag_FEMME.csv",sep=",")
df_nike_mag_Homme = pd.read_csv("data_NikeMag_HOMME.csv",sep=",")
df_nike_mag_Enfant = pd.read_csv("data_NikeMag_ENFANT.csv",sep=",")

In [17]:
df_nike_mag = pd.concat([df_nike_mag_Femme, df_nike_mag_Homme, df_nike_mag_Enfant], axis=0, ignore_index=True)
df_nike_mag

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,Nike Blazer,7.5,NIKE,"BLANC, CORAIL",38.5,https://www.vinted.fr/items/3808211046-nike-bl...,https://images1.vinted.net/t/02_01ee5_jJaEXHni...,"ROTTERDAM, PAYS-BAS",Women
1,Sneakers - Nike Roshe One Valentine's Day,10.0,NIKE,"BLANC, ROUGE",37.5,https://www.vinted.fr/items/3883913822-sneaker...,https://images1.vinted.net/t/02_0130f_JDdNTYHx...,PAYS-BAS,Women
2,Prachtige Air force 1 by Nike,20.0,NIKE,"BLANC, MULTICOLORE",38.5,https://www.vinted.fr/items/3859318004-prachti...,https://images1.vinted.net/t/03_00385_43gpofFH...,PAYS-BAS,Women
3,Nike sneakers,30.0,NIKE,"BLANC, BEIGE",38.5,https://www.vinted.fr/items/3894770172-nike-sn...,https://images1.vinted.net/t/02_00c06_X6wpDDpS...,BELGIQUE,Women
4,Nike sportschoenen,20.0,NIKE,"BLANC, CRÈME",37.5,https://www.vinted.fr/items/3894774215-nike-sp...,https://images1.vinted.net/t/01_01b79_yhUBrBWD...,BELGIQUE,Women
...,...,...,...,...,...,...,...,...,...
875,Nike schoenen,30.0,NIKE,"NOIR, BLANC",36,https://www.vinted.fr/items/1236922541-nike-sc...,https://images1.vinted.net/t/03_01742_NN736grX...,"SILVOLDE, PAYS-BAS",Kid
876,"Nike air max 19,5",20.0,NIKE,"BLEU, MARINE",19,https://www.vinted.fr/items/2171498196-nike-ai...,https://images1.vinted.net/t/03_00e8e_Kz6UUsCV...,PAYS-BAS,Kid
877,Nike mt 25,15.0,NIKE,MULTICOLORE,25,https://www.vinted.fr/items/328396909-nike-mt-...,https://images1.vinted.net/t/01_005a8_pnV9Qp3J...,"ROTTERDAM, PAYS-BAS",Kid
878,Nike city of dreams maat 18.5,75.0,NIKE,MULTICOLORE,18,https://www.vinted.fr/items/3408759047-nike-ci...,https://images1.vinted.net/t/01_0169c_ryiDKJkd...,"DE MEERN, PAYS-BAS",Kid


In [18]:
# Adding columns to the DataFrame for the approximate carbon footprint and Nike's fixed value
df_nike_mag["Approximate Carbon Footprint (kg CO2)"] = df_nike_mag["Localisation"].apply(calculate_approximate_carbon_footprint)
df_nike_mag["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] = 405  

# Adding a column for Carbon Profit
df_nike_mag["Carbon Profit (kg CO2)"] = df_nike_mag["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] - df_nike_mag["Approximate Carbon Footprint (kg CO2)"]

df_nike_mag

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2)
0,Nike Blazer,7.5,NIKE,"BLANC, CORAIL",38.5,https://www.vinted.fr/items/3808211046-nike-bl...,https://images1.vinted.net/t/02_01ee5_jJaEXHni...,"ROTTERDAM, PAYS-BAS",Women,74.638487,405,330.361513
1,Sneakers - Nike Roshe One Valentine's Day,10.0,NIKE,"BLANC, ROUGE",37.5,https://www.vinted.fr/items/3883913822-sneaker...,https://images1.vinted.net/t/02_0130f_JDdNTYHx...,PAYS-BAS,Women,87.928759,405,317.071241
2,Prachtige Air force 1 by Nike,20.0,NIKE,"BLANC, MULTICOLORE",38.5,https://www.vinted.fr/items/3859318004-prachti...,https://images1.vinted.net/t/03_00385_43gpofFH...,PAYS-BAS,Women,87.928759,405,317.071241
3,Nike sneakers,30.0,NIKE,"BLANC, BEIGE",38.5,https://www.vinted.fr/items/3894770172-nike-sn...,https://images1.vinted.net/t/02_00c06_X6wpDDpS...,BELGIQUE,Women,51.833601,405,353.166399
4,Nike sportschoenen,20.0,NIKE,"BLANC, CRÈME",37.5,https://www.vinted.fr/items/3894774215-nike-sp...,https://images1.vinted.net/t/01_01b79_yhUBrBWD...,BELGIQUE,Women,51.833601,405,353.166399
...,...,...,...,...,...,...,...,...,...,...,...,...
875,Nike schoenen,30.0,NIKE,"NOIR, BLANC",36,https://www.vinted.fr/items/1236922541-nike-sc...,https://images1.vinted.net/t/03_01742_NN736grX...,"SILVOLDE, PAYS-BAS",Kid,88.958335,405,316.041665
876,"Nike air max 19,5",20.0,NIKE,"BLEU, MARINE",19,https://www.vinted.fr/items/2171498196-nike-ai...,https://images1.vinted.net/t/03_00e8e_Kz6UUsCV...,PAYS-BAS,Kid,87.928759,405,317.071241
877,Nike mt 25,15.0,NIKE,MULTICOLORE,25,https://www.vinted.fr/items/328396909-nike-mt-...,https://images1.vinted.net/t/01_005a8_pnV9Qp3J...,"ROTTERDAM, PAYS-BAS",Kid,74.638487,405,330.361513
878,Nike city of dreams maat 18.5,75.0,NIKE,MULTICOLORE,18,https://www.vinted.fr/items/3408759047-nike-ci...,https://images1.vinted.net/t/01_0169c_ryiDKJkd...,"DE MEERN, PAYS-BAS",Kid,81.221083,405,323.778917


In [19]:
url4 = "https://www.nike.com/fr/w?q=nike%20mag&vst=nike%20mag"
average_nike_price4=get_nike_prices(url4)
average_nike_price4

102.83600000000001

In [20]:
df_nike_mag.sort_values(by='Price', inplace=True)

# Calculate profit made for Vinted data
df_nike_mag['Average price of the product on Nike'] = average_nike_price4
df_nike_mag['Profit Made'] = df_nike_mag.apply(lambda row: calculate_profit_made(row, average_nike_price4), axis=1)

df_nike_mag

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
824,Nike schoenen,5.0,NIKE,"GRIS, ROSE",36,https://www.vinted.fr/items/487316313-nike-sch...,https://images1.vinted.net/t/01_018a6_YouAt8aC...,"SINT-OEDENRODE, PAYS-BAS",Kid,74.813111,405,330.186889,102.836,97.836
817,Nike star runner lichtblauw maat 26,5.0,NIKE,BLEU CLAIR,26,https://www.vinted.fr/items/3759104432-nike-st...,https://images1.vinted.net/t/02_00b26_5A5aoVEL...,"APELDOORN, PAYS-BAS",Kid,89.647343,405,315.352657,102.836,97.836
24,Zwarte Nike - maat 38,5.0,NIKE,"NOIR, BLANC",38,https://www.vinted.fr/items/3660145888-zwarte-...,https://images1.vinted.net/t/02_007b4_UkQoacaM...,"HAMME, BELGIQUE",Women,55.744253,405,349.255747,102.836,97.836
816,Nike air Max maat 21,5.0,NIKE,"BLANC, VIOLET",21,https://www.vinted.fr/items/3030702278-nike-ai...,https://images1.vinted.net/t/03_00d4f_ry6CtrXX...,PAYS-BAS,Kid,87.928759,405,317.071241,102.836,97.836
810,Leuke gympen,5.0,NIKE,"GRIS, MULTICOLORE",38,https://www.vinted.fr/items/2646578334-leuke-g...,https://images1.vinted.net/t/03_00351_JdzuqXb5...,"ROTTERDAM, PAYS-BAS",Kid,74.638487,405,330.361513,102.836,97.836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,Nike Vapor Max,NaN,"150,00 €",GRIS,Nike,https://www.vinted.fr/items/594540023-nike-vap...,https://images1.vinted.net/t/01_01fc1_zZ5SzEjx...,"MAASTRICHT, PAYS-BAS",Men,65.532110,405,339.467890,102.836,NaN
786,Nike dunk low oranje groen,NaN,"100,00 €","ORANGE, VERT",Nike,https://www.vinted.fr/items/3406528522-nike-du...,https://images1.vinted.net/t/02_01abb_DcRYwZSH...,PAYS-BAS,Men,87.928759,405,317.071241,102.836,NaN
793,Nike air max 720,NaN,"109,95 €","NOIR, MULTICOLORE",Nike,https://www.vinted.fr/items/936910666-nike-air...,https://images1.vinted.net/t/02_00dba_ywGsAULy...,BELGIQUE,Men,51.833601,405,353.166399,102.836,NaN
812,Nooit gedragen,NaN,"50,00 €",BLANC,Nike,https://www.vinted.fr/items/3850412938-nooit-g...,https://images1.vinted.net/t/01_014fe_tXhw4WEK...,PAYS-BAS,Kid,87.928759,405,317.071241,102.836,NaN


In [21]:
df_nike_mag= df_nike_mag.dropna(subset=['Profit Made'])
df_nike_mag

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
824,Nike schoenen,5.0,NIKE,"GRIS, ROSE",36,https://www.vinted.fr/items/487316313-nike-sch...,https://images1.vinted.net/t/01_018a6_YouAt8aC...,"SINT-OEDENRODE, PAYS-BAS",Kid,74.813111,405,330.186889,102.836,97.836
817,Nike star runner lichtblauw maat 26,5.0,NIKE,BLEU CLAIR,26,https://www.vinted.fr/items/3759104432-nike-st...,https://images1.vinted.net/t/02_00b26_5A5aoVEL...,"APELDOORN, PAYS-BAS",Kid,89.647343,405,315.352657,102.836,97.836
24,Zwarte Nike - maat 38,5.0,NIKE,"NOIR, BLANC",38,https://www.vinted.fr/items/3660145888-zwarte-...,https://images1.vinted.net/t/02_007b4_UkQoacaM...,"HAMME, BELGIQUE",Women,55.744253,405,349.255747,102.836,97.836
816,Nike air Max maat 21,5.0,NIKE,"BLANC, VIOLET",21,https://www.vinted.fr/items/3030702278-nike-ai...,https://images1.vinted.net/t/03_00d4f_ry6CtrXX...,PAYS-BAS,Kid,87.928759,405,317.071241,102.836,97.836
810,Leuke gympen,5.0,NIKE,"GRIS, MULTICOLORE",38,https://www.vinted.fr/items/2646578334-leuke-g...,https://images1.vinted.net/t/03_00351_JdzuqXb5...,"ROTTERDAM, PAYS-BAS",Kid,74.638487,405,330.361513,102.836,97.836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,Nike Air Max 1 Psychic Pink Deadstock EU44,400.0,NIKE,ROSE,44,https://www.vinted.fr/items/3836206670-nike-ai...,https://images1.vinted.net/t/03_0168f_6F5sWbZr...,"MAASTRICHT, PAYS-BAS",Men,65.532110,405,339.467890,102.836,-297.164
673,Nike Air Max 1 x Travis Scott Cactus Jack Baro...,430.0,NIKE,"BEIGE, MARRON","42,5",https://www.vinted.fr/items/3899122736-nike-ai...,https://images1.vinted.net/t/03_00d82_wW3iPnHS...,"'S-GRAVENHAGE, PAYS-BAS",Men,76.911773,405,328.088227,102.836,-327.164
589,Nike adapt BB 2.0 mag,500.0,NIKE,"BLANC, TURQUOISE",42,https://www.vinted.fr/items/3905835169-nike-ad...,https://images1.vinted.net/t/01_0013b_8yze2qyf...,"ZEMST, BELGIQUE",Men,56.217774,405,348.782226,102.836,-397.164
719,nike bb mag,500.0,NIKE,GRIS,44,https://www.vinted.fr/items/471726506-nike-bb-...,https://images1.vinted.net/t/02_00512_Zse5NQ3h...,"ARMANCOURT, FRANCE",Men,19.006560,405,385.993440,102.836,-397.164


In [22]:
df_nike_mag.to_csv("AllData_NikeMag.csv",index=False)

### Nike Air Max 1 

##### Femme

In [17]:
url = "https://www.vinted.fr/catalog?search_text=nike%20air%20max%201&brand_ids[]=53&catalog[]=2632&status_ids[]=6&status_ids[]=1&status_ids[]=2"
vinted_nike_airmax1_femme = get_vinted_data(url,400)
df_nike_airmax1_Femme = process_dataframe(vinted_nike_airmax1_femme)
df_nike_airmax1_Femme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Nike air max,49.0,NIKE,"NOIR, GRIS",38.5,https://www.vinted.fr/items/3839052451-nike-ai...,https://images1.vinted.net/t/01_000a1_5YNP536M...,"NIEUWKOOP, PAYS-BAS"
1,Nike air max 1,25.0,NIKE,"BLANC, MARINE",36.5,https://www.vinted.fr/items/3817987665-nike-ai...,https://images1.vinted.net/t/03_0205c_vvkQfQSm...,FRANCE
2,Air Max 1,50.0,NIKE,"BLANC, JAUNE",38,https://www.vinted.fr/items/3831072468-air-max...,https://images1.vinted.net/t/01_023da_oqeYbKZA...,"CHÂTEAU-THIERRY, FRANCE"
3,Nike air Max 1,50.0,NIKE,"BLANC, ROUGE",38.5,https://www.vinted.fr/items/3847225041-nike-ai...,https://images1.vinted.net/t/02_011b9_iTZ5W6oE...,"PARIS, FRANCE"
4,Basket nike air max,25.0,NIKE,NOIR,38.5,https://www.vinted.fr/items/3872678741-basket-...,https://images1.vinted.net/t/02_011a1_xFMET8vD...,FRANCE
...,...,...,...,...,...,...,...,...
395,Nike Air Max 1 Safari Summit White,120.0,NIKE,"BLANC, BEIGE",36.5,https://www.vinted.fr/items/3903374543-nike-ai...,https://images1.vinted.net/t/03_01af4_5VGJoh6m...,"ALKMAAR, PAYS-BAS"
396,Air max 1 maat 40,80.0,NIKE,NOIR,40,https://www.vinted.fr/items/3918742045-air-max...,https://images1.vinted.net/t/02_017b7_8RHYUSfj...,"BREDA, PAYS-BAS"
397,Nike Air Max 1 Just Do It Pack White,200.0,NIKE,"BLANC, ORANGE",40.5,https://www.vinted.fr/items/3907412934-nike-ai...,https://images1.vinted.net/t/01_01c34_2MNpexgE...,"RUNGIS, FRANCE"
398,Air max 1 nike,21.0,NIKE,"BLANC, BLEU",37.5,https://www.vinted.fr/items/2590498254-air-max...,https://images1.vinted.net/t/01_012e8_Xa1JCy1S...,"AULNOYE-AYMERIES, FRANCE"


In [18]:
df_nike_airmax1_Femme['Gender']="Women"

In [19]:
df_nike_airmax1_Femme.to_csv("data_NikeAirmax1_FEMME.csv",index=False)

##### Homme

In [20]:
url = "https://www.vinted.fr/catalog?search_text=nike%20air%20max%201&brand_ids[]=53&catalog[]=1242&status_ids[]=6&status_ids[]=1&status_ids[]=2"
vinted_nike_airmax1_homme = get_vinted_data(url,400)
df_nike_airmax1_Homme = process_dataframe(vinted_nike_airmax1_homme)
df_nike_airmax1_Homme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Baskets Air Max T42,35.0,NIKE,"NOIR, BLANC",42,https://www.vinted.fr/items/3918986877-baskets...,https://images1.vinted.net/t/02_00b9e_qFNveQS5...,"BRESSE VALLONS, FRANCE"
1,Air max 1,30.0,NIKE,VERT,44,https://www.vinted.fr/items/3683530690-air-max...,https://images1.vinted.net/t/03_00313_a9H5shJF...,"COURLAY, FRANCE"
2,Air Max One Trainers Mide,49.0,NIKE,"GRIS, BLANC","40,5",https://www.vinted.fr/items/3919014448-air-max...,https://images1.vinted.net/t/02_01222_JxqKJkLy...,"LUTILHOUS, FRANCE"
3,"Obra ""Nike AIR Max _ 1/97 """,8.0,NIKE,"BLANC, MULTICOLORE",44,https://www.vinted.fr/items/3905942860-obra-ni...,https://images1.vinted.net/t/01_014c3_yKAPeDDp...,ESPAGNE
4,"Air Max 1 pale Ivory taille 42,5",100.0,NIKE,"GRIS, BEIGE","42,5",https://www.vinted.fr/items/3877055734-air-max...,https://images1.vinted.net/t/01_00354_yat8AvU7...,"ARNOUVILLE, FRANCE"
...,...,...,...,...,...,...,...,...
395,air max,50.0,NIKE,"NOIR, ROSE",39,https://www.vinted.fr/items/3824285178-air-max...,https://images1.vinted.net/t/02_01e8a_u62JA3va...,FRANCE
396,Nike Air max 97 taille 41,55.0,NIKE,"NOIR, BLANC",41,https://www.vinted.fr/items/3891010149-nike-ai...,https://images1.vinted.net/t/02_02550_MHmRAnyL...,"VIGNEUX-SUR-SEINE, FRANCE"
397,Nike Air Max Plus TN Blanche Beige Homme,125.0,NIKE,"BLANC, BEIGE",47,https://www.vinted.fr/items/3906081847-nike-ai...,https://images1.vinted.net/t/01_02222_godDawYu...,FRANCE
398,nike air max 1 clot kiss of death,145.0,NIKE,"BEIGE, ABRICOT",41,https://www.vinted.fr/items/3905686260-nike-ai...,https://images1.vinted.net/t/02_00754_GJteVqtW...,FRANCE


In [21]:
df_nike_airmax1_Homme['Gender']="Men"

In [22]:
df_nike_airmax1_Homme.to_csv("data_NikeAirmax1_HOMME.csv",index=False)

##### Enfant

In [24]:
url = "https://www.vinted.fr/catalog?search_text=nike%20air%20max%201&brand_ids[]=53&catalog[]=1255&status_ids[]=6&status_ids[]=1&status_ids[]=2"
vinted_nike_airmax1_enfant = get_vinted_data(url,100)
df_nike_airmax1_Enfant = process_dataframe(vinted_nike_airmax1_enfant)
df_nike_airmax1_Enfant

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,Nike Air Max 1 TD maat 25,20.0,NIKE,"GRIS, ROSE",25,https://www.vinted.fr/items/3879168745-nike-ai...,https://images1.vinted.net/t/02_01b0b_9T7mMpit...,PAYS-BAS
1,Basket air max 1,8.0,NIKE,"BLANC, ROSE",29,https://www.vinted.fr/items/3839735280-basket-...,https://images1.vinted.net/t/03_00008_kVVmSUMY...,"ROCHE-LA-MOLIÈRE, FRANCE"
2,Nike air max 27,25.0,NIKE,BLANC,27,https://www.vinted.fr/items/3881059153-nike-ai...,https://images1.vinted.net/t/03_017d1_CAQo45dG...,"SAINT-VENANT, FRANCE"
3,Basket Nike air max plus Tuned 1 black neuves ...,28.0,NIKE,NOIR,23,https://www.vinted.fr/items/3781348723-basket-...,https://images1.vinted.net/t/03_01087_eS36Gp66...,"NÎMES, FRANCE"
4,Air max,15.0,NIKE,"GRIS, BLANC",25,https://www.vinted.fr/items/3907447187-air-max...,https://images1.vinted.net/t/01_020f0_sNd95jik...,FRANCE
...,...,...,...,...,...,...,...,...
95,Basket nike air max,70.0,NIKE,"NOIR, ROSE",36,https://www.vinted.fr/items/487316009-basket-n...,https://images1.vinted.net/t/01_01f3f_MxQVjz7P...,"GRASSE, FRANCE"
96,Air max,20.0,NIKE,"GRIS, ROSE",19,https://www.vinted.fr/items/979422675-air-max?...,https://images1.vinted.net/t/02_00244_EN6GbUws...,"CIRAL, FRANCE"
97,Nike air max one 1,90.0,NIKE,"ROUGE, BLANC",39,https://www.vinted.fr/items/3199270675-nike-ai...,https://images1.vinted.net/t/02_00bd8_en3qP2Gs...,"GEX, FRANCE"
98,Basket nike air max,70.0,NIKE,"NOIR, ROSE",36,https://www.vinted.fr/items/487316009-basket-n...,https://images1.vinted.net/t/01_01f3f_MxQVjz7P...,"GRASSE, FRANCE"


In [25]:
df_nike_airmax1_Enfant['Gender']="Kid"

In [26]:
df_nike_airmax1_Enfant.to_csv("data_NikeAirmax1_ENFANT.csv",index=False)

##### Concataned Dataframe of Nike Air Max 1

In [1]:
import pandas as pd
df_nike_airmax1_Femme = pd.read_csv("data_NikeAirmax1_FEMME.csv",sep=",")
df_nike_airmax1_Homme = pd.read_csv("data_NikeAirmax1_HOMME.csv",sep=",")
df_nike_airmax1_Enfant = pd.read_csv("data_NikeAirmax1_ENFANT.csv",sep=",")

In [3]:
df_nike_airmax1 = pd.concat([df_nike_airmax1_Femme, df_nike_airmax1_Homme, df_nike_airmax1_Enfant], axis=0, ignore_index=True)
df_nike_airmax1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,Nike air max,49.0,NIKE,"NOIR, GRIS",38.5,https://www.vinted.fr/items/3839052451-nike-ai...,https://images1.vinted.net/t/01_000a1_5YNP536M...,"NIEUWKOOP, PAYS-BAS",Women
1,Nike air max 1,25.0,NIKE,"BLANC, MARINE",36.5,https://www.vinted.fr/items/3817987665-nike-ai...,https://images1.vinted.net/t/03_0205c_vvkQfQSm...,FRANCE,Women
2,Air Max 1,50.0,NIKE,"BLANC, JAUNE",38,https://www.vinted.fr/items/3831072468-air-max...,https://images1.vinted.net/t/01_023da_oqeYbKZA...,"CHÂTEAU-THIERRY, FRANCE",Women
3,Nike air Max 1,50.0,NIKE,"BLANC, ROUGE",38.5,https://www.vinted.fr/items/3847225041-nike-ai...,https://images1.vinted.net/t/02_011b9_iTZ5W6oE...,"PARIS, FRANCE",Women
4,Basket nike air max,25.0,NIKE,NOIR,38.5,https://www.vinted.fr/items/3872678741-basket-...,https://images1.vinted.net/t/02_011a1_xFMET8vD...,FRANCE,Women
...,...,...,...,...,...,...,...,...,...
895,Basket nike air max,70.0,NIKE,"NOIR, ROSE",36,https://www.vinted.fr/items/487316009-basket-n...,https://images1.vinted.net/t/01_01f3f_MxQVjz7P...,"GRASSE, FRANCE",Kid
896,Air max,20.0,NIKE,"GRIS, ROSE",19,https://www.vinted.fr/items/979422675-air-max?...,https://images1.vinted.net/t/02_00244_EN6GbUws...,"CIRAL, FRANCE",Kid
897,Nike air max one 1,90.0,NIKE,"ROUGE, BLANC",39,https://www.vinted.fr/items/3199270675-nike-ai...,https://images1.vinted.net/t/02_00bd8_en3qP2Gs...,"GEX, FRANCE",Kid
898,Basket nike air max,70.0,NIKE,"NOIR, ROSE",36,https://www.vinted.fr/items/487316009-basket-n...,https://images1.vinted.net/t/01_01f3f_MxQVjz7P...,"GRASSE, FRANCE",Kid


In [8]:
# Adding columns to the DataFrame for the approximate carbon footprint and Nike's fixed value
df_nike_airmax1["Approximate Carbon Footprint (kg CO2)"] = df_nike_airmax1["Localisation"].apply(calculate_approximate_carbon_footprint)
df_nike_airmax1["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] = 405  

# Adding a column for Carbon Profit
df_nike_airmax1["Carbon Profit (kg CO2)"] = df_nike_airmax1["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] - df_nike_airmax1["Approximate Carbon Footprint (kg CO2)"]

df_nike_airmax1


,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2)
0,Nike air max,49.0,NIKE,"NOIR, GRIS",38.5,https://www.vinted.fr/items/3839052451-nike-ai...,https://images1.vinted.net/t/01_000a1_5YNP536M...,"NIEUWKOOP, PAYS-BAS",Women,80.931706,405,324.068294
1,Nike air max 1,25.0,NIKE,"BLANC, MARINE",36.5,https://www.vinted.fr/items/3817987665-nike-ai...,https://images1.vinted.net/t/03_0205c_vvkQfQSm...,FRANCE,Women,50.586158,405,354.413842
2,Air Max 1,50.0,NIKE,"BLANC, JAUNE",38,https://www.vinted.fr/items/3831072468-air-max...,https://images1.vinted.net/t/01_023da_oqeYbKZA...,"CHÂTEAU-THIERRY, FRANCE",Women,15.953058,405,389.046942
3,Nike air Max 1,50.0,NIKE,"BLANC, ROUGE",38.5,https://www.vinted.fr/items/3847225041-nike-ai...,https://images1.vinted.net/t/02_011b9_iTZ5W6oE...,"PARIS, FRANCE",Women,0.474704,405,404.525296
4,Basket nike air max,25.0,NIKE,NOIR,38.5,https://www.vinted.fr/items/3872678741-basket-...,https://images1.vinted.net/t/02_011a1_xFMET8vD...,FRANCE,Women,50.586158,405,354.413842
...,...,...,...,...,...,...,...,...,...,...,...,...
895,Basket nike air max,70.0,NIKE,"NOIR, ROSE",36,https://www.vinted.fr/items/487316009-basket-n...,https://images1.vinted.net/t/01_01f3f_MxQVjz7P...,"GRASSE, FRANCE",Kid,135.300290,405,269.699710
896,Air max,20.0,NIKE,"GRIS, ROSE",19,https://www.vinted.fr/items/979422675-air-max?...,https://images1.vinted.net/t/02_00244_EN6GbUws...,"CIRAL, FRANCE",Kid,37.546656,405,367.453344
897,Nike air max one 1,90.0,NIKE,"ROUGE, BLANC",39,https://www.vinted.fr/items/3199270675-nike-ai...,https://images1.vinted.net/t/02_00bd8_en3qP2Gs...,"GEX, FRANCE",Kid,79.060332,405,325.939668
898,Basket nike air max,70.0,NIKE,"NOIR, ROSE",36,https://www.vinted.fr/items/487316009-basket-n...,https://images1.vinted.net/t/01_01f3f_MxQVjz7P...,"GRASSE, FRANCE",Kid,135.300290,405,269.699710


In [9]:
url5 = "https://www.nike.com/fr/w?q=nike%20AIR%20MAX%201&vst=nike%20AIR%20MAX%201"
average_nike_price5=get_nike_prices(url5)
average_nike_price5

133.50124999999994

In [12]:
df_nike_airmax1.sort_values(by='Price', inplace=True)

# Calculate profit made for Vinted data
df_nike_airmax1['Average price of the product on Nike'] = average_nike_price5
df_nike_airmax1['Profit Made'] = df_nike_airmax1.apply(lambda row: calculate_profit_made(row, average_nike_price5), axis=1)

df_nike_airmax1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
835,Air max taille 21,8.0,NIKE,"GRIS, JAUNE",21,https://www.vinted.fr/items/3678325504-air-max...,https://images1.vinted.net/t/02_00d84_3Wu7J6o1...,FRANCE,Kid,50.586158,405,354.413842,133.50125,125.50125
801,Basket air max 1,8.0,NIKE,"BLANC, ROSE",29,https://www.vinted.fr/items/3839735280-basket-...,https://images1.vinted.net/t/03_00008_kVVmSUMY...,"ROCHE-LA-MOLIÈRE, FRANCE",Kid,81.726695,405,323.273305,133.50125,125.50125
20,"Basket Nike ""Air Max""",8.0,NIKE,"BLANC, ROSE",35,https://www.vinted.fr/items/3904312378-basket-...,https://images1.vinted.net/t/01_0027d_5fVmJV7V...,"MENNECY, FRANCE",Women,6.561751,405,398.438249,133.50125,125.50125
806,Sapatilhas Nike Air Max tamanho 23 1/2,8.0,NIKE,"GRIS, ROSE",23,https://www.vinted.fr/items/2987733294-sapatil...,https://images1.vinted.net/t/02_011ef_4kBjaDBz...,"LOBÃO, PORTUGAL",Kid,244.273365,405,160.726635,133.50125,125.50125
403,"Obra ""Nike AIR Max _ 1/97 """,8.0,NIKE,"BLANC, MULTICOLORE",44,https://www.vinted.fr/items/3905942860-obra-ni...,https://images1.vinted.net/t/01_014c3_yKAPeDDp...,ESPAGNE,Men,240.819687,405,164.180313,133.50125,125.50125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,"Nike air max 1 ""shima shima""",NaN,NaN,"GRIS, BLANC","110,00 €",https://www.vinted.fr/items/3899530913-nike-ai...,https://images1.vinted.net/t/01_00949_NkdcGFsa...,"ALBISOLA SUPERIORE, ITALIE",Women,137.705232,405,267.294768,133.50125,NaN
284,Nike air max 1 'blue void',NaN,"69,00 €","BLANC, BLEU",Nike,https://www.vinted.fr/items/3918299950-nike-ai...,https://images1.vinted.net/t/02_01764_EmsDugdr...,"ALMELO, PAYS-BAS",Women,98.939512,405,306.060488,133.50125,NaN
693,Nike Air Max 1 x Lebron James x Liverpool F.C ...,NaN,"179,99 €","NOIR, VERT FONCÉ",Nike,https://www.vinted.fr/items/3914213922-nike-ai...,https://images1.vinted.net/t/02_01005_LEycfcg3...,"SAINT-YAGUEN, FRANCE",Men,120.231290,405,284.768710,133.50125,NaN
826,Air,NaN,"25,00 €",NOIR,Nike,https://www.vinted.fr/items/3815620787-air-max...,https://images1.vinted.net/t/03_00a2b_Ug4GJwz4...,"CARS, FRANCE",Kid,94.385014,405,310.614986,133.50125,NaN


In [13]:
df_nike_airmax1= df_nike_airmax1.dropna(subset=['Profit Made'])
df_nike_airmax1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
835,Air max taille 21,8.0,NIKE,"GRIS, JAUNE",21,https://www.vinted.fr/items/3678325504-air-max...,https://images1.vinted.net/t/02_00d84_3Wu7J6o1...,FRANCE,Kid,50.586158,405,354.413842,133.50125,125.50125
801,Basket air max 1,8.0,NIKE,"BLANC, ROSE",29,https://www.vinted.fr/items/3839735280-basket-...,https://images1.vinted.net/t/03_00008_kVVmSUMY...,"ROCHE-LA-MOLIÈRE, FRANCE",Kid,81.726695,405,323.273305,133.50125,125.50125
20,"Basket Nike ""Air Max""",8.0,NIKE,"BLANC, ROSE",35,https://www.vinted.fr/items/3904312378-basket-...,https://images1.vinted.net/t/01_0027d_5fVmJV7V...,"MENNECY, FRANCE",Women,6.561751,405,398.438249,133.50125,125.50125
806,Sapatilhas Nike Air Max tamanho 23 1/2,8.0,NIKE,"GRIS, ROSE",23,https://www.vinted.fr/items/2987733294-sapatil...,https://images1.vinted.net/t/02_011ef_4kBjaDBz...,"LOBÃO, PORTUGAL",Kid,244.273365,405,160.726635,133.50125,125.50125
403,"Obra ""Nike AIR Max _ 1/97 """,8.0,NIKE,"BLANC, MULTICOLORE",44,https://www.vinted.fr/items/3905942860-obra-ni...,https://images1.vinted.net/t/01_014c3_yKAPeDDp...,ESPAGNE,Men,240.819687,405,164.180313,133.50125,125.50125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,Nike Air Max 1 Parra,600.0,NIKE,"BLANC, BLEU","44,5",https://www.vinted.fr/items/3912851424-nike-ai...,https://images1.vinted.net/t/01_00035_HfH5mpwx...,PAYS-BAS,Men,87.928759,405,317.071241,133.50125,-466.49875
492,Nike Air Max 1 Premium Atmos Elephant,650.0,NIKE,"NOIR, TURQUOISE","44,5",https://www.vinted.fr/items/3912906186-nike-ai...,https://images1.vinted.net/t/01_0250f_kket2ygi...,PAYS-BAS,Men,87.928759,405,317.071241,133.50125,-516.49875
710,Nike Air Max 1 Amsterdam,650.0,NIKE,"BEIGE, ABRICOT","44,5",https://www.vinted.fr/items/3913016719-nike-ai...,https://images1.vinted.net/t/02_01046_jhd3fTEc...,PAYS-BAS,Men,87.928759,405,317.071241,133.50125,-516.49875
525,Air max 1 denham,690.0,NIKE,MARINE,"45,5",https://www.vinted.fr/items/3918356848-air-max...,https://images1.vinted.net/t/01_00e0d_TqcXtgFX...,FRANCE,Men,50.586158,405,354.413842,133.50125,-556.49875


In [15]:
df_nike_airmax1.to_csv("AllData_NikeAirmax1.csv",index=False)